# Scrape Forum Data
This notebook scrapes the Netmums and inserts the results into a SQLite database

## Data Sources
- Netmums forum: https://www.netmums.com/coffeehouse/

## Changes
- 2020-01-30: Created
- 2020-02-02: Added thread scraping
- 2020-02-05: Moved functions to netmums.py

## TODO
- 

## Imports

In [3]:
import sqlite3
from pathlib import Path
from netmums import *
from scraping import *
from datetime import datetime
import os
import csv
import re
from tqdm import tqdm

## File Locations

In [2]:
p = Path.cwd()
path_parent = p.parents[0]

In [3]:
path_db = str(path_parent / "database" / "netmums-merged.db")
path_groups = str(path_parent / "scripts" / "netmums-group_{}.py")

## Get Forum List

Create list of Netmum forum pages in the db. Skip if already created. 

In [4]:
conn = create_connection(path_db)
cur = conn.cursor()

In [5]:
cur.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='forums' ''')
if cur.fetchone()[0] == 0:
    set_up_merged_db(conn)

In [6]:
cur.execute(''' SELECT count(id) FROM forums ''')
if cur.fetchone()[0] == 0:
    url = "https://www.netmums.com/coffeehouse/"
    soup = get_soup(url)
    forums = soup.find_all("div", {"class": "left"})
    for forum in forums:
        if forum_link := forum.find('a', href=True):
            parsed = (forum_link['href'].strip(), forum_link.get_text().strip())
            sql = '''
                INSERT INTO forums(url,name)
                VALUES(?,?)
            '''
            cur.execute(sql, parsed)
conn.commit()

## Subforums

In [7]:
cur.execute(''' SELECT count(id) FROM subforums ''')
if cur.fetchone()[0] == 0:
    cur.execute(''' SELECT id, url FROM forums ''')
    rows = cur.fetchall()
    for row in rows:
        (forum_id, url) = row
        soup = get_soup(url)
        for subforum in soup.find_all("a", {"class": "cCatTopic"}):
            parsed = (subforum['href'].strip(), subforum.get_text().strip(),forum_id)
            sql = '''
                INSERT INTO subforums(url,name,forum_id)
                VALUES(?,?,?)
            '''
            cur.execute(sql, parsed)
conn.commit()

## Threads

In [8]:
cur.execute(''' SELECT count(id) FROM threads ''')
if cur.fetchone()[0] == 0:
    cur.execute(''' SELECT id, url FROM subforums ''')
    rows = cur.fetchall()
    for row in tqdm(rows):
        scrape_threads(cur, row)
        conn.commit()

## Posts
Create individual scraper files for each chunk of thread urls. Run with: ```nohup python3 netmums-group_xx.py > output_xx.txt &```

Posts can have different formatting in different situations, which leads to problems with parsing:
1. Must parse posts with citations to other posts differently than those without citations
2. Long urls are shortened when posted directly
3. WYSIWYG emojis must be translated from image to text
4. Skip the first post on each page after the first, it is duplicated from the last

This can be restart automatically. It deletes the last record collected in threads and starts rescraping that thread_id (if one exists).

In [9]:
cur.execute(''' SELECT max(id) FROM threads ''')
max_id = cur.fetchone()[0]

In [10]:
n_groups = 5
size = round(max_id / n_groups)

In [11]:
text = """#!/usr/bin/env python3
# coding: utf-8

## Imports

from netmums import *
from scraping import *
from pathlib import Path

## File Locations

p = Path.cwd()
path_parent = p.parents[0]
path_db_parent = str(path_parent / "database" / "netmums-merged.db")
path_db_child = str(path_parent / "database" / "netmums{0}.db")

## Connect to the database and create the tables.

### find max id in child db posts
conn = create_connection(path_db_child)
set_up_posts_db(conn)
cur = conn.cursor()
cur.execute(''' SELECT MAX(thread_id) FROM posts ''')
max_thread = cur.fetchone()[0]
if max_thread == None:
    first = {1}
else: # restart scraping from last complete thread id
    cur.execute(''' DELETE FROM posts WHERE thread_id=?''', (max_thread,))
    conn.commit()
    first = max_thread
conn.close()

### select rows from parent db threads
conn = create_connection(path_db_parent)
cur = conn.cursor()
cur.execute(''' SELECT id, url FROM threads WHERE id>=? AND id<={2} ''', (first,))
rows = cur.fetchall()
conn.close()

### connect back to child db
conn = create_connection(path_db_child)
set_up_posts_db(conn)

## Scrape threads

for row in rows:
    scrape_posts(conn, row)

conn.close()
"""


In [12]:
for g in range(n_groups):
    first = 1 + (size * g)
    if g == (n_groups - 1):
        last = max_id
    else:
        last = (size * (g + 1))
    if g + 1 < 10:
        num = "0{}".format(g + 1)
    else:
        num = "{}".format(g + 1)
    with open(path_groups.format(num), 'w') as w:
        w.write(text.format(num, first, last))

In [13]:
conn.close()

In [25]:
html="""<!DOCTYPE html>
<html lang="en"><head><link href="https://fonts.googleapis.com" rel="preconnect"/><link href="https://fonts.gstatic.com" rel="preconnect"/><link href="https://fonts.googleapis.com/css?family=Amatic+SC:700%7CRoboto:400,500,700&amp;display=swap" media="all" rel="stylesheet"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><meta content="563844106964124" property="fb:app_id"/><meta content="Netmums" property="og:site_name"/><meta content="https://www.netmums.com/site/assets/images/meta_chat.png" property="og:image"/><meta content="1200" property="og:image:width"/><meta content="630" property="og:image:height"/><meta content="@netmums" name="twitter:site"/><meta content="summary_large_image" name="twitter:card"/><meta content="https://www.netmums.com/site/assets/images/meta_chat.png" name="twitter:image"/><link href="https://www.netmums.com/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/><link href="https://www.netmums.com/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="https://www.netmums.com/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link href="https://www.netmums.com/site.webmanifest" rel="manifest"/><link color="#be1f4b" href="https://www.netmums.com/safari-pinned-tab.svg" rel="mask-icon"/><meta content="#be1f4b" name="msapplication-TileColor"/><meta content="#ffffff" name="theme-color"/><meta content="Netmums" name="apple-mobile-web-app-title"/><meta content="Netmums" name="application-name"/><meta content="app-id=634874784" name="apple-itunes-app"/><link href="https://www.netmums.com/min/g=mes-discussions.css&amp;1606922973" rel="stylesheet" type="text/css"/><script lang="js">window.gdprAppliesGlobally=true;</script><title>To all you lovely savvy shoppers</title><meta content="noindex" name="robots"/><meta content="For all of you who hold your breath at the checkout waiting for the final total of your shopping:shock:  We are a family of 7 myself,OH,3DS's 14,12 and 9 and 2 DD's 7 and 3.  We used to spend £680 a month at Asda and unbeleaveably we'd still have days where we'd say we have'nt got anything for dinner:shock:" name="description"/><link href="https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-688.html" rel="canonical"/><script async="" type="application/ld+json">{"@context":"https://schema.org","@type":"DiscussionForumPosting","@id":"https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-688.html","headline":"To all you lovely savvy shoppers","author":{"@type":"Person","name":"Anonymous"},"publisher":{"@type":"Person","name":"Anonymous"},"datePublished":"2020-10-19T22:46:50Z","image":"https://www.netmums.com/favicon-32x32.png","interactionStatistic":{"@type":"InteractionCounter","interactionType":"https://schema.org/CommentAction","userInteractionCount":14300}}</script><link href="https://pegasus.unifygroup.com" rel="preconnect"/><script async="">var pegasusConf = {"params":{"site":"netmums","traffic":"organic","country":"UK","keywords":"user=false;section=moneyadvice;forum=609f,budgeting-609;thread=551294t;url=/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers.html;content_keywords=To,all,you,lovely,savvy,shoppers","section":"forum","contenttype":"forum"}}</script><script async="" src="https://pegasus.unifygroup.com/lib/pegasus.min.js" type="text/javascript"></script><script type="application/ld+json">{"@context":"https://schema.org","@type":"BreadcrumbList","itemListElement":[{"@type":"ListItem","position":1,"item":{"@id":"https://www.netmums.com/coffeehouse/money-advice-1127/","name":"Money advice"}},{"@type":"ListItem","position":2,"item":{"@id":"https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/","name":"Budgeting"}}]}</script><meta content="31" name="next-head-count"/><noscript data-n-css=""></noscript><link as="script" href="/coffeehouse/_next/static/chunks/19.690c57d0f5640b429bd3.js" rel="preload"/><link as="script" href="/coffeehouse/_next/static/chunks/commons.f0b5f4e603dc3a39ca88.js" rel="preload"/><link as="script" href="/coffeehouse/_next/static/chunks/c8f7fe3b0e41be846d5687592cf2018ff6e22687.43470669db83827181f2.js" rel="preload"/><link as="script" href="/coffeehouse/_next/static/chunks/14.8eaaf22aed8fbe6f30e4.js" rel="preload"/><link as="script" href="/coffeehouse/_next/static/chunks/main-d7fbbbb2a64728a34fdf.js" rel="preload"/><link as="script" href="/coffeehouse/_next/static/chunks/webpack-d622334fbd1a1df22771.js" rel="preload"/><link as="script" href="/coffeehouse/_next/static/chunks/framework.8a2b607e5c9e7ff716ca.js" rel="preload"/><link as="script" href="/coffeehouse/_next/static/chunks/commons.f0b5f4e603dc3a39ca88.js" rel="preload"/><link as="script" href="/coffeehouse/_next/static/chunks/5ae162f7fb127fc582bc2fc041b9ecba1a2752e0.0264b613bc7ccc9e2c8b.js" rel="preload"/><link as="script" href="/coffeehouse/_next/static/chunks/pages/_app-3129c050fdf6796067b0.js" rel="preload"/><link as="script" href="/coffeehouse/_next/static/chunks/pages/%5B...slugs%5D-badbd68de45e1929b348.js" rel="preload"/><style id="jss-server-side">.MuiCollapse-container {
  height: 0;
  overflow: hidden;
  transition: height 300ms cubic-bezier(0.4, 0, 0.2, 1) 0ms;
}
.MuiCollapse-entered {
  height: auto;
  overflow: visible;
}
.MuiCollapse-hidden {
  visibility: hidden;
}
.MuiCollapse-wrapper {
  display: flex;
}
.MuiCollapse-wrapperInner {
  width: 100%;
}
.MuiButtonBase-root {
  color: inherit;
  border: 0;
  cursor: pointer;
  margin: 0;
  display: inline-flex;
  outline: 0;
  padding: 0;
  position: relative;
  align-items: center;
  user-select: none;
  border-radius: 0;
  vertical-align: middle;
  -moz-appearance: none;
  justify-content: center;
  text-decoration: none;
  background-color: transparent;
  -webkit-appearance: none;
  -webkit-tap-highlight-color: transparent;
}
.MuiButtonBase-root::-moz-focus-inner {
  border-style: none;
}
.MuiButtonBase-root.Mui-disabled {
  cursor: default;
  pointer-events: none;
}
@media print {
  .MuiButtonBase-root {
    color-adjust: exact;
  }
}
  .MuiIconButton-root {
    flex: 0 0 auto;
    color: rgba(0, 0, 0, 0.54);
    padding: 12px;
    overflow: visible;
    font-size: 1.5rem;
    text-align: center;
    transition: background-color 150ms cubic-bezier(0.4, 0, 0.2, 1) 0ms;
    border-radius: 50%;
  }
  .MuiIconButton-root:hover {
    background-color: rgba(0, 0, 0, 0.04);
  }
  .MuiIconButton-root.Mui-disabled {
    color: rgba(0, 0, 0, 0.26);
    background-color: transparent;
  }
@media (hover: none) {
  .MuiIconButton-root:hover {
    background-color: transparent;
  }
}
  .MuiIconButton-edgeStart {
    margin-left: -12px;
  }
  .MuiIconButton-sizeSmall.MuiIconButton-edgeStart {
    margin-left: -3px;
  }
  .MuiIconButton-edgeEnd {
    margin-right: -12px;
  }
  .MuiIconButton-sizeSmall.MuiIconButton-edgeEnd {
    margin-right: -3px;
  }
  .MuiIconButton-colorInherit {
    color: inherit;
  }
  .MuiIconButton-colorPrimary {
    color: #3f51b5;
  }
  .MuiIconButton-colorPrimary:hover {
    background-color: rgba(63, 81, 181, 0.04);
  }
@media (hover: none) {
  .MuiIconButton-colorPrimary:hover {
    background-color: transparent;
  }
}
  .MuiIconButton-colorSecondary {
    color: #f50057;
  }
  .MuiIconButton-colorSecondary:hover {
    background-color: rgba(245, 0, 87, 0.04);
  }
@media (hover: none) {
  .MuiIconButton-colorSecondary:hover {
    background-color: transparent;
  }
}
  .MuiIconButton-sizeSmall {
    padding: 3px;
    font-size: 1.125rem;
  }
  .MuiIconButton-label {
    width: 100%;
    display: flex;
    align-items: inherit;
    justify-content: inherit;
  }
  .MuiAvatar-root {
    width: 40px;
    height: 40px;
    display: flex;
    overflow: hidden;
    position: relative;
    font-size: 1.25rem;
    align-items: center;
    flex-shrink: 0;
    font-family: "Helvetica",Arial,sans-serif;
    line-height: 1;
    user-select: none;
    border-radius: 50%;
    justify-content: center;
  }
  .MuiAvatar-colorDefault {
    color: #e0e0e0;
    background-color: #bdbdbd;
  }
  .MuiAvatar-circle {
    color: #2A2A2A;
    width: 100%;
    height: 100%;
  }
  .MuiAvatar-rounded {
    border-radius: 4px;
  }
  .MuiAvatar-square {
    border-radius: 0;
  }
  .MuiAvatar-img {
    color: transparent;
    width: 100%;
    height: 100%;
    object-fit: cover;
    text-align: center;
    text-indent: 10000px;
  }
  .MuiAvatar-fallback {
    width: 75%;
    height: 75%;
  }
  .MuiTypography-root {
    margin: 0;
  }
  .MuiTypography-body2 {
    font-size: 0.875rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 400;
    line-height: 1.43;
  }
  .MuiTypography-body1 {
    font-size: 1rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 400;
    line-height: 1.5;
  }
  .MuiTypography-caption {
    font-size: 0.75rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 400;
    line-height: 1.66;
  }
  .MuiTypography-button {
    font-size: 0.875rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 500;
    line-height: 1.75;
    text-transform: uppercase;
  }
  .MuiTypography-h1 {
    font-size: 6rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 300;
    line-height: 1.167;
  }
  .MuiTypography-h2 {
    font-size: 3.75rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 300;
    line-height: 1.2;
  }
  .MuiTypography-h3 {
    font-size: 3rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 400;
    line-height: 1.167;
  }
  .MuiTypography-h4 {
    font-size: 2.125rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 400;
    line-height: 1.235;
  }
  .MuiTypography-h5 {
    font-size: 1.5rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 400;
    line-height: 1.334;
  }
  .MuiTypography-h6 {
    font-size: 1.25rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 500;
    line-height: 1.6;
  }
  .MuiTypography-subtitle1 {
    font-size: 1rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 400;
    line-height: 1.75;
  }
  .MuiTypography-subtitle2 {
    font-size: 0.875rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 500;
    line-height: 1.57;
  }
  .MuiTypography-overline {
    font-size: 0.75rem;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 400;
    line-height: 2.66;
    text-transform: uppercase;
  }
  .MuiTypography-srOnly {
    width: 1px;
    height: 1px;
    overflow: hidden;
    position: absolute;
  }
  .MuiTypography-alignLeft {
    text-align: left;
  }
  .MuiTypography-alignCenter {
    text-align: center;
  }
  .MuiTypography-alignRight {
    text-align: right;
  }
  .MuiTypography-alignJustify {
    text-align: justify;
  }
  .MuiTypography-noWrap {
    overflow: hidden;
    white-space: nowrap;
    text-overflow: ellipsis;
  }
  .MuiTypography-gutterBottom {
    margin-bottom: 0.35em;
  }
  .MuiTypography-paragraph {
    margin-bottom: 16px;
  }
  .MuiTypography-colorInherit {
    color: inherit;
  }
  .MuiTypography-colorPrimary {
    color: #3f51b5;
  }
  .MuiTypography-colorSecondary {
    color: #f50057;
  }
  .MuiTypography-colorTextPrimary {
    color: rgba(0, 0, 0, 0.87);
  }
  .MuiTypography-colorTextSecondary {
    color: rgba(0, 0, 0, 0.54);
  }
  .MuiTypography-colorError {
    color: #f44336;
  }
  .MuiTypography-displayInline {
    display: inline;
  }
  .MuiTypography-displayBlock {
    display: block;
  }
  .MuiBreadcrumbs-ol {
    margin: 0;
    display: flex;
    padding: 0;
    flex-wrap: wrap;
    list-style: none;
    align-items: center;
  }
  .MuiBreadcrumbs-separator {
    display: flex;
    margin-left: 8px;
    user-select: none;
    margin-right: 8px;
  }
  .MuiButton-root {
    color: rgba(0, 0, 0, 0.87);
    padding: 5px;
    font-size: 16px;
    min-width: 64px;
    box-sizing: border-box;
    transition: background-color 250ms cubic-bezier(0.4, 0, 0.2, 1) 0ms,box-shadow 250ms cubic-bezier(0.4, 0, 0.2, 1) 0ms,border 250ms cubic-bezier(0.4, 0, 0.2, 1) 0ms;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 500;
    line-height: normal;
    border-radius: 8px;
    text-transform: none;
  }
  .MuiButton-root:hover {
    text-decoration: none;
    background-color: rgba(0, 0, 0, 0.04);
  }
  .MuiButton-root.Mui-disabled {
    color: rgba(0, 0, 0, 0.26);
  }
@media (hover: none) {
  .MuiButton-root:hover {
    background-color: transparent;
  }
}
  .MuiButton-root:hover.Mui-disabled {
    background-color: transparent;
  }
  .MuiButton-label {
    width: 100%;
    display: inherit;
    align-items: inherit;
    justify-content: inherit;
  }
  .MuiButton-text {
    color: #2A2A2A;
    padding: 6px 8px;
  }
  .MuiButton-textPrimary {
    color: #2A2A2A;
  }
  .MuiButton-textPrimary:hover {
    background-color: rgba(63, 81, 181, 0.04);
  }
@media (hover: none) {
  .MuiButton-textPrimary:hover {
    background-color: transparent;
  }
}
  .MuiButton-textSecondary {
    color: #a3a3a3;
  }
  .MuiButton-textSecondary:hover {
    background-color: rgba(245, 0, 87, 0.04);
  }
@media (hover: none) {
  .MuiButton-textSecondary:hover {
    background-color: transparent;
  }
}
  .MuiButton-outlined {
    border: 1px solid;
    padding: 5px 15px;
    border-color: #2A2A2A;
  }
  .MuiButton-outlined.Mui-disabled {
    border: 1px solid rgba(0, 0, 0, 0.12);
  }
  .MuiButton-outlinedPrimary {
    color: #e6005a;
    border: 1px solid rgba(63, 81, 181, 0.5);
    border-color: #e6005a;
  }
  .MuiButton-outlinedPrimary:hover {
    color: #FFFFFF;
    border: 1px solid #3f51b5;
    border-color: #e6005a;
    background-color: #e6005a;
  }
  .MuiButton-outlinedPrimary:disabled {
    color: #111111;
    opacity: 0.5;
    border-color: #111111;
    background-color: #f0f0f0;
  }
@media (hover: none) {
  .MuiButton-outlinedPrimary:hover {
    color: #FFFFFF;
    border-color: #e6005a;
    background-color: #e6005a;
  }
}
  .MuiButton-outlinedSecondary {
    color: #f50057;
    border: 1px solid rgba(245, 0, 87, 0.5);
  }
  .MuiButton-outlinedSecondary:hover {
    border: 1px solid #f50057;
    background-color: rgba(245, 0, 87, 0.04);
  }
  .MuiButton-outlinedSecondary.Mui-disabled {
    border: 1px solid rgba(0, 0, 0, 0.26);
  }
@media (hover: none) {
  .MuiButton-outlinedSecondary:hover {
    background-color: transparent;
  }
}
  .MuiButton-contained {
    color: #2A2A2A;
    box-shadow: 0px 3px 1px -2px rgba(0,0,0,0.2),0px 2px 2px 0px rgba(0,0,0,0.14),0px 1px 5px 0px rgba(0,0,0,0.12);
    background-color: #F8F8F8;
  }
  .MuiButton-contained:hover {
    box-shadow: 0px 2px 4px -1px rgba(0,0,0,0.2),0px 4px 5px 0px rgba(0,0,0,0.14),0px 1px 10px 0px rgba(0,0,0,0.12);
    background-color: #F8F8F8;
  }
  .MuiButton-contained.Mui-focusVisible {
    box-shadow: 0px 3px 5px -1px rgba(0,0,0,0.2),0px 6px 10px 0px rgba(0,0,0,0.14),0px 1px 18px 0px rgba(0,0,0,0.12);
  }
  .MuiButton-contained:active {
    box-shadow: 0px 5px 5px -3px rgba(0,0,0,0.2),0px 8px 10px 1px rgba(0,0,0,0.14),0px 3px 14px 2px rgba(0,0,0,0.12);
  }
  .MuiButton-contained.Mui-disabled {
    color: rgba(0, 0, 0, 0.26);
    box-shadow: none;
    background-color: rgba(0, 0, 0, 0.12);
  }
@media (hover: none) {
  .MuiButton-contained:hover {
    box-shadow: 0px 3px 1px -2px rgba(0,0,0,0.2),0px 2px 2px 0px rgba(0,0,0,0.14),0px 1px 5px 0px rgba(0,0,0,0.12);
    background-color: #F8F8F8;
  }
}
  .MuiButton-contained:hover.Mui-disabled {
    background-color: rgba(0, 0, 0, 0.12);
  }
  .MuiButton-containedPrimary {
    color: #fff;
    background-color: #3f51b5;
  }
  .MuiButton-containedPrimary:hover {
    background-color: #303f9f;
  }
@media (hover: none) {
  .MuiButton-containedPrimary:hover {
    background-color: #3f51b5;
  }
}
  .MuiButton-containedSecondary {
    color: #2A2A2A;
    background-color: #f7bf3c;
  }
  .MuiButton-containedSecondary:hover {
    background-color: #dca72c;
  }
  .MuiButton-containedSecondary:disabled {
    color: #2A2A2A;
    background-color: #d5d5d5;
  }
@media (hover: none) {
  .MuiButton-containedSecondary:hover {
    background-color: #dca72c;
  }
}
  .MuiButton-disableElevation {
    box-shadow: none;
  }
  .MuiButton-disableElevation:hover {
    box-shadow: none;
  }
  .MuiButton-disableElevation.Mui-focusVisible {
    box-shadow: none;
  }
  .MuiButton-disableElevation:active {
    box-shadow: none;
  }
  .MuiButton-disableElevation.Mui-disabled {
    box-shadow: none;
  }
  .MuiButton-colorInherit {
    color: inherit;
    border-color: currentColor;
  }
  .MuiButton-textSizeSmall {
    padding: 4px 5px;
    font-size: 0.8125rem;
  }
  .MuiButton-textSizeLarge {
    padding: 8px 11px;
    font-size: 0.9375rem;
  }
  .MuiButton-outlinedSizeSmall {
    padding: 3px 9px;
    font-size: 0.8125rem;
  }
  .MuiButton-outlinedSizeLarge {
    padding: 7px 21px;
    font-size: 0.9375rem;
  }
  .MuiButton-containedSizeSmall {
    padding: 4px 10px;
    font-size: 0.8125rem;
  }
  .MuiButton-containedSizeLarge {
    padding: 8px 22px;
    font-size: 0.9375rem;
  }
  .MuiButton-sizeSmall {
    width: auto;
    padding: 8px 11px;
    font-size: 14px;
    border-radius: 8px;
  }
  .MuiButton-fullWidth {
    width: 100%;
    padding: 10px;
    border-radius: 25px;
  }
  .MuiButton-startIcon {
    display: inherit;
    margin-left: -4px;
    margin-right: 8px;
  }
  .MuiButton-startIcon.MuiButton-iconSizeSmall {
    margin-left: -2px;
  }
  .MuiButton-endIcon {
    display: inherit;
    margin-left: 8px;
    margin-right: -4px;
  }
  .MuiButton-endIcon.MuiButton-iconSizeSmall {
    margin-right: -2px;
  }
  .MuiButton-iconSizeSmall {
    width: 12px;
    height: 12px;
  }
  .MuiButton-iconSizeSmall > *:first-child {
    font-size: 18px;
  }
  .MuiButton-iconSizeMedium > *:first-child {
    font-size: 20px;
  }
  .MuiButton-iconSizeLarge > *:first-child {
    font-size: 22px;
  }
  .MuiLink-root {
    color: #2A2A2A;
  }
  .MuiLink-underlineNone {
    text-decoration: none;
  }
  .MuiLink-underlineHover {
    text-decoration: none;
  }
  .MuiLink-underlineHover:hover {
    text-decoration: underline;
  }
  .MuiLink-underlineAlways {
    text-decoration: underline;
  }
  .MuiLink-button {
    border: 0;
    cursor: pointer;
    margin: 0;
    outline: 0;
    padding: 0;
    position: relative;
    user-select: none;
    border-radius: 0;
    vertical-align: middle;
    -moz-appearance: none;
    background-color: transparent;
    -webkit-appearance: none;
    -webkit-tap-highlight-color: transparent;
  }
  .MuiLink-button::-moz-focus-inner {
    border-style: none;
  }
  .MuiLink-button.Mui-focusVisible {
    outline: auto;
  }
  .MuiPopover-paper {
    outline: 0;
    position: absolute;
    max-width: calc(100% - 32px);
    min-width: 16px;
    box-shadow: 0px 2px 4px rgba(164, 164, 164, 0.13);
    max-height: calc(100% - 32px);
    min-height: 16px;
    overflow-x: hidden;
    overflow-y: auto;
    border-radius: 13px;
  }
  .MuiTooltip-popper {
    z-index: 1500;
    pointer-events: none;
  }
  .MuiTooltip-popperInteractive {
    pointer-events: auto;
  }
  .MuiTooltip-popperArrow[x-placement*="bottom"] .MuiTooltip-arrow {
    top: 0;
    left: 0;
    margin-top: -0.71em;
    margin-left: 4px;
    margin-right: 4px;
  }
  .MuiTooltip-popperArrow[x-placement*="top"] .MuiTooltip-arrow {
    left: 0;
    bottom: 0;
    margin-left: 4px;
    margin-right: 4px;
    margin-bottom: -0.71em;
  }
  .MuiTooltip-popperArrow[x-placement*="right"] .MuiTooltip-arrow {
    left: 0;
    width: 0.71em;
    height: 1em;
    margin-top: 4px;
    margin-left: -0.71em;
    margin-bottom: 4px;
  }
  .MuiTooltip-popperArrow[x-placement*="left"] .MuiTooltip-arrow {
    right: 0;
    width: 0.71em;
    height: 1em;
    margin-top: 4px;
    margin-right: -0.71em;
    margin-bottom: 4px;
  }
  .MuiTooltip-popperArrow[x-placement*="left"] .MuiTooltip-arrow::before {
    transform-origin: 0 0;
  }
  .MuiTooltip-popperArrow[x-placement*="right"] .MuiTooltip-arrow::before {
    transform-origin: 100% 100%;
  }
  .MuiTooltip-popperArrow[x-placement*="top"] .MuiTooltip-arrow::before {
    transform-origin: 100% 0;
  }
  .MuiTooltip-popperArrow[x-placement*="bottom"] .MuiTooltip-arrow::before {
    transform-origin: 0 100%;
  }
  .MuiTooltip-tooltip {
    color: #fff;
    padding: 4px 8px;
    font-size: 0.625rem;
    max-width: 300px;
    word-wrap: break-word;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 500;
    line-height: 1.4em;
    border-radius: 4px;
    background-color: rgba(97, 97, 97, 0.9);
  }
  .MuiTooltip-tooltipArrow {
    margin: 0;
    position: relative;
  }
  .MuiTooltip-arrow {
    color: rgba(97, 97, 97, 0.9);
    width: 1em;
    height: 0.71em;
    overflow: hidden;
    position: absolute;
    box-sizing: border-box;
  }
  .MuiTooltip-arrow::before {
    width: 100%;
    height: 100%;
    margin: auto;
    content: "";
    display: block;
    transform: rotate(45deg);
    background-color: currentColor;
  }
  .MuiTooltip-touch {
    padding: 8px 16px;
    font-size: 0.875rem;
    font-weight: 400;
    line-height: 1.14286em;
  }
  .MuiTooltip-tooltipPlacementLeft {
    margin: 0 24px ;
    transform-origin: right center;
  }
@media (min-width:600px) {
  .MuiTooltip-tooltipPlacementLeft {
    margin: 0 14px;
  }
}
  .MuiTooltip-tooltipPlacementRight {
    margin: 0 24px;
    transform-origin: left center;
  }
@media (min-width:600px) {
  .MuiTooltip-tooltipPlacementRight {
    margin: 0 14px;
  }
}
  .MuiTooltip-tooltipPlacementTop {
    margin: 24px 0;
    transform-origin: center bottom;
  }
@media (min-width:600px) {
  .MuiTooltip-tooltipPlacementTop {
    margin: 14px 0;
  }
}
  .MuiTooltip-tooltipPlacementBottom {
    margin: 24px 0;
    transform-origin: center top;
  }
@media (min-width:600px) {
  .MuiTooltip-tooltipPlacementBottom {
    margin: 14px 0;
  }
}
  .MuiPaginationItem-root {
    color: #2A2A2A;
    height: 32px;
    margin: 0 3px;
    padding: 0 6px;
    font-size: 0.875rem;
    min-width: 32px;
    box-sizing: border-box;
    text-align: center;
    font-family: "Helvetica",Arial,sans-serif;
    font-weight: 400;
    line-height: 1.43;
    border-radius: 16px;
  }
  .MuiPaginationItem-page {
    transition: color 250ms cubic-bezier(0.4, 0, 0.2, 1) 0ms,background-color 250ms cubic-bezier(0.4, 0, 0.2, 1) 0ms;
  }
  .MuiPaginationItem-page:hover {
    background-color: rgba(0, 0, 0, 0.04);
  }
  .MuiPaginationItem-page.Mui-focusVisible {
    background-color: rgba(0, 0, 0, 0.12);
  }
  .MuiPaginationItem-page.Mui-selected {
    background-color: rgba(0, 0, 0, 0.08);
  }
  .MuiPaginationItem-page.Mui-disabled {
    opacity: 0.38;
  }
  .MuiPaginationItem-page.Mui-selected:hover, .MuiPaginationItem-page.Mui-selected.Mui-focusVisible {
    background-color: rgba(0, 0, 0, 0.12);
  }
  .MuiPaginationItem-page.Mui-selected.Mui-disabled {
    color: rgba(0, 0, 0, 0.26);
    opacity: 1;
    background-color: rgba(0, 0, 0, 0.08);
  }
@media (hover: none) {
  .MuiPaginationItem-page.Mui-selected:hover, .MuiPaginationItem-page.Mui-selected.Mui-focusVisible {
    background-color: rgba(0, 0, 0, 0.08);
  }
}
@media (hover: none) {
  .MuiPaginationItem-page:hover {
    background-color: transparent;
  }
}
  .MuiPaginationItem-sizeSmall {
    height: 26px;
    margin: 0 1px;
    padding: 0 4px;
    min-width: 26px;
    border-radius: 13px;
  }
  .MuiPaginationItem-sizeSmall .MuiPaginationItem-icon {
    font-size: 1.125rem;
  }
  .MuiPaginationItem-sizeLarge {
    height: 40px;
    padding: 0 10px;
    font-size: 0.9375rem;
    min-width: 40px;
    border-radius: 20px;
  }
  .MuiPaginationItem-sizeLarge .MuiPaginationItem-icon {
    font-size: 1.375rem;
  }
  .MuiPaginationItem-textPrimary {
    color: #2A2A2A;
    background-color: #FFFFFF;
  }
  .MuiPaginationItem-textPrimary.Mui-selected {
    color: #FFFFFF;
    background-color: #e6005a;
  }
@media (hover: none) {
  .MuiPaginationItem-textPrimary:hover!.Mui-selected {
    color: #e6005a;
  }
}
  .MuiPaginationItem-textPrimary.Mui-selected:hover, .MuiPaginationItem-textPrimary.Mui-selected.Mui-focusVisible {
    background-color: #303f9f;
  }
  .MuiPaginationItem-textPrimary.Mui-selected.Mui-disabled {
    color: rgba(0, 0, 0, 0.26);
  }
@media (hover: none) {
  .MuiPaginationItem-textPrimary.Mui-selected:hover, .MuiPaginationItem-textPrimary.Mui-selected.Mui-focusVisible {
    background-color: #3f51b5;
  }
}
  .MuiPaginationItem-textSecondary.Mui-selected {
    color: #fff;
    background-color: #f50057;
  }
  .MuiPaginationItem-textSecondary.Mui-selected:hover, .MuiPaginationItem-textSecondary.Mui-selected.Mui-focusVisible {
    background-color: #c51162;
  }
  .MuiPaginationItem-textSecondary.Mui-selected.Mui-disabled {
    color: rgba(0, 0, 0, 0.26);
  }
@media (hover: none) {
  .MuiPaginationItem-textSecondary.Mui-selected:hover, .MuiPaginationItem-textSecondary.Mui-selected.Mui-focusVisible {
    background-color: #f50057;
  }
}
  .MuiPaginationItem-outlined {
    border: 1px solid rgba(0, 0, 0, 0.23);
  }
  .MuiPaginationItem-outlined.Mui-selected.Mui-disabled {
    border: 1px solid rgba(0, 0, 0, 0.12);
  }
  .MuiPaginationItem-outlinedPrimary.Mui-selected {
    color: #3f51b5;
    border: 1px solid rgba(63, 81, 181, 0.5);
    background-color: rgba(63, 81, 181, 0.12);
  }
  .MuiPaginationItem-outlinedPrimary.Mui-selected:hover, .MuiPaginationItem-outlinedPrimary.Mui-selected.Mui-focusVisible {
    background-color: rgba(63, 81, 181, 0.16);
  }
  .MuiPaginationItem-outlinedPrimary.Mui-selected.Mui-disabled {
    color: rgba(0, 0, 0, 0.26);
  }
@media (hover: none) {
  .MuiPaginationItem-outlinedPrimary.Mui-selected:hover, .MuiPaginationItem-outlinedPrimary.Mui-selected.Mui-focusVisible {
    background-color: transparent;
  }
}
  .MuiPaginationItem-outlinedSecondary.Mui-selected {
    color: #f50057;
    border: 1px solid rgba(245, 0, 87, 0.5);
    background-color: rgba(245, 0, 87, 0.12);
  }
  .MuiPaginationItem-outlinedSecondary.Mui-selected:hover, .MuiPaginationItem-outlinedSecondary.Mui-selected.Mui-focusVisible {
    background-color: rgba(245, 0, 87, 0.16);
  }
  .MuiPaginationItem-outlinedSecondary.Mui-selected.Mui-disabled {
    color: rgba(0, 0, 0, 0.26);
  }
@media (hover: none) {
  .MuiPaginationItem-outlinedSecondary.Mui-selected:hover, .MuiPaginationItem-outlinedSecondary.Mui-selected.Mui-focusVisible {
    background-color: transparent;
  }
}
  .MuiPaginationItem-rounded {
    border-radius: 4px;
  }
  .MuiPaginationItem-ellipsis {
    height: auto;
  }
  .MuiPaginationItem-ellipsis.Mui-disabled {
    opacity: 0.38;
  }
  .MuiPaginationItem-icon {
    margin: 0 -8px;
    font-size: 1.25rem;
  }</style><style data-styled="" data-styled-version="5.2.1">html,body,div,span,applet,object,iframe,h1,h2,h3,h4,h5,h6,p,blockquote,pre,a,abbr,acronym,address,big,cite,code,del,dfn,em,img,ins,kbd,q,s,samp,small,strike,strong,sub,sup,tt,var,b,u,i,center,dl,dt,dd,ol,ul,li,fieldset,form,label,legend,table,caption,tbody,tfoot,thead,tr,th,td,article,aside,canvas,details,embed,figure,figcaption,footer,header,hgroup,menu,nav,output,ruby,section,summary,time,mark,audio,video{margin:0;padding:0;font:inherit;font-size:100%;vertical-align:baseline;border:0;}/*!sc*/
article,aside,details,figcaption,figure,footer,header,hgroup,menu,nav,section{display:block;}/*!sc*/
ol,ul{list-style:none;}/*!sc*/
blockquote,q{quotes:none;}/*!sc*/
blockquote::before,blockquote::after,q::before,q::after{content:'';content:none;}/*!sc*/
table{border-collapse:collapse;border-spacing:0;}/*!sc*/
*{box-sizing:border-box;}/*!sc*/
html{height:100vh;color:#2A2A2A;font-weight:normal;font-size:16px;font-family:'Roboto','Helvetica','Arial',sans-serif;line-height:22px;}/*!sc*/
body,#__next{height:100%;}/*!sc*/
input{border:none;outline:none;}/*!sc*/
button{-webkit-tap-highlight-color:transparent;}/*!sc*/
data-styled.g1[id="sc-global-kMctFN1"]{content:"sc-global-kMctFN1,"}/*!sc*/
.cpmnmV{height:100%;margin:auto;padding:0;}/*!sc*/
@media (min-width:1024px){.cpmnmV{width:1000px;}}/*!sc*/
data-styled.g2[id="Rootstyle__RootContainer-sc-1faqzk7-0"]{content:"cpmnmV,"}/*!sc*/
.igzlTH{-webkit-box-flex:1;-webkit-flex-grow:1;-ms-flex-positive:1;flex-grow:1;border-radius:13px;}/*!sc*/
data-styled.g3[id="Rootstyle__PageContent-sc-1faqzk7-1"]{content:"igzlTH,"}/*!sc*/
.eFYlhl{width:100%;margin-bottom:25px;}/*!sc*/
data-styled.g4[id="BannerAtfstyle__BannerAtfContainer-sc-1eqlkbi-0"]{content:"eFYlhl,"}/*!sc*/
.eVhCqB{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;margin-bottom:30px;padding:0 30px;text-align:center;}/*!sc*/
.eVhCqB > *:not(:last-child){margin-bottom:30px;}/*!sc*/
data-styled.g5[id="SearchCtastyle__Container-sc-123tvxn-0"]{content:"eVhCqB,"}/*!sc*/
.iModOL{width:315px;}/*!sc*/
data-styled.g6[id="SearchCtastyle__Content-sc-123tvxn-1"]{content:"iModOL,"}/*!sc*/
.iKSYmX{border-color:#2A2A2A;background-color:#f7bf3c;disabled:#d5d5d5;hover-background-color:#dca72c;font-weight:normal;font-size:24px;font-family:'Amatic SC','Roboto','Helvetica','Arial',sans-serif;}/*!sc*/
.iKSYmX:hover{border-color:#2A2A2A;background-color:#f7bf3c;disabled:#d5d5d5;hover-background-color:#dca72c;}/*!sc*/
data-styled.g7[id="SearchCtastyle__SearchThreadButton-sc-123tvxn-2"]{content:"iKSYmX,"}/*!sc*/
.awMNh{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;text-align:center;background-color:#F6F7F1;border-radius:13px;box-shadow:0px 2px 4px rgba(164,164,164,0.13);}/*!sc*/
data-styled.g8[id="NewThreadCtastyle__Container-il2ii3-0"]{content:"awMNh,"}/*!sc*/
.efiGch{margin-bottom:25px;font-weight:500;font-size:28px;line-height:38px;}/*!sc*/
data-styled.g9[id="NewThreadCtastyle__CustomTitleContainer-il2ii3-1"]{content:"efiGch,"}/*!sc*/
.laLnJE{color:#FFFFFF;font-weight:normal;font-size:24px;font-family:'Amatic SC','Roboto','Helvetica','Arial',sans-serif;background-color:#e6005a;}/*!sc*/
.laLnJE:hover{background-color:#e6005a;}/*!sc*/
data-styled.g10[id="NewThreadCtastyle__CreateThreadButton-il2ii3-2"]{content:"laLnJE,"}/*!sc*/
.lhTTCP{max-width:335px;padding-top:30px;padding-bottom:30px;}/*!sc*/
data-styled.g11[id="NewThreadCtastyle__Content-il2ii3-3"]{content:"lhTTCP,"}/*!sc*/
.eKVeqz{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;background-color:#F8F8F8;}/*!sc*/
data-styled.g12[id="Videostyle__Container-jonus6-0"]{content:"eKVeqz,"}/*!sc*/
.ccJNiz{z-index:999;}/*!sc*/
data-styled.g13[id="Videostyle__VideoContainer-jonus6-1"]{content:"ccJNiz,"}/*!sc*/
.gOHhMk{margin-bottom:30px;text-align:center;}/*!sc*/
data-styled.g14[id="SeeLastAnswerCtastyle__Container-sc-1lpozhn-0"]{content:"gOHhMk,"}/*!sc*/
.kiAQko{color:#2A2A2A;font-weight:normal;font-size:16px;font-family:'Roboto','Helvetica','Arial',sans-serif;line-height:22px;}/*!sc*/
.kiAQko:hover{color:#FFFFFF;background-color:#e6005a;border-color:#e6005a;}/*!sc*/
data-styled.g15[id="SeeLastAnswerCtastyle__SeeLastAnswerButton-sc-1lpozhn-1"]{content:"kiAQko,"}/*!sc*/
.btLhUS{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-box-pack:center;-webkit-justify-content:center;-ms-flex-pack:center;justify-content:center;}/*!sc*/
data-styled.g22[id="CustomTitlestyle__TitleContainer-sc-112n8kf-0"]{content:"btLhUS,"}/*!sc*/
.bqCWeV{-webkit-align-content:center;-ms-flex-line-pack:center;align-content:center;width:10px;height:40px;}/*!sc*/
data-styled.g23[id="CustomTitlestyle__TitleDecoration-sc-112n8kf-1"]{content:"bqCWeV,"}/*!sc*/
.igONYw{width:10px;}/*!sc*/
data-styled.g24[id="CustomTitlestyle__TitleDecorationImg-sc-112n8kf-2"]{content:"igONYw,"}/*!sc*/
.daRUhb{margin:0 15px;font-size:38px;text-align:center;}/*!sc*/
data-styled.g25[id="CustomTitlestyle__Title-sc-112n8kf-3"]{content:"daRUhb,"}/*!sc*/
.kBLUtW{margin-bottom:30px;padding-top:15px;padding-bottom:15px;background-color:#F6F7F1;}/*!sc*/
data-styled.g26[id="SimilarThreadsstyle__SimilarThreadsContainer-m9qzem-0"]{content:"kBLUtW,"}/*!sc*/
.cTZebo{padding:10px;font-family:'Amatic SC','Roboto','Helvetica','Arial',sans-serif;}/*!sc*/
data-styled.g27[id="SimilarThreadsstyle__TitleContainer-m9qzem-1"]{content:"cTZebo,"}/*!sc*/
.hgblsD{color:inherit;-webkit-text-decoration:none;text-decoration:none;}/*!sc*/
data-styled.g28[id="SimilarThreadsstyle__UnstyledLink-m9qzem-2"]{content:"hgblsD,"}/*!sc*/
.XnEW{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;height:100%;padding:15px 0;overflow-x:auto;}/*!sc*/
data-styled.g29[id="SimilarThreadsstyle__SimilarThreadsSlider-m9qzem-3"]{content:"XnEW,"}/*!sc*/
.bKzkOx{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;box-sizing:border-box;min-width:min(330px,70vw);max-width:min(330px,70vw);height:100%;margin:0 15px;padding:15px;background-color:#FFFFFF;border-radius:18px;}/*!sc*/
data-styled.g30[id="SimilarThreadsstyle__SimilarThreadCard-m9qzem-4"]{content:"bKzkOx,"}/*!sc*/
.bZfCKE{-webkit-box-flex:1;-webkit-flex-grow:1;-ms-flex-positive:1;flex-grow:1;margin-bottom:10px;font-size:16px;-webkit-text-decoration:none;text-decoration:none;}/*!sc*/
data-styled.g31[id="SimilarThreadsstyle__SimilarThreadCardTitle-m9qzem-5"]{content:"bZfCKE,"}/*!sc*/
.cqHQJQ{margin-bottom:10px;color:#419B9B;font-weight:600;font-size:14px;}/*!sc*/
data-styled.g32[id="SimilarThreadsstyle__SimilarThreadCategory-m9qzem-6"]{content:"cqHQJQ,"}/*!sc*/
.fmAZHQ{background-color:#F8F8F8;}/*!sc*/
.fmAZHQ .pegasus_status_empty{display:none;}/*!sc*/
.fmAZHQ .pegasus_status_rendered{margin-bottom:30px;}/*!sc*/
data-styled.g33[id="InterPostComponentstyle__AdBannerContainer-e122kk-0"]{content:"fmAZHQ,"}/*!sc*/
.bLGqZf{margin-bottom:30px;margin-right:0;margin-left:0;}/*!sc*/
data-styled.g34[id="InterPostComponentstyle__MarginableContainer-e122kk-1"]{content:"bLGqZf,"}/*!sc*/
.cQGPpo{width:55px;height:55px;background-color:transparent;}/*!sc*/
data-styled.g35[id="Avatarstyle__CustomAvatar-sc-1x0bfr6-0"]{content:"cQGPpo,"}/*!sc*/
.cdFmmG{width:100%;height:100%;}/*!sc*/
data-styled.g36[id="Avatarstyle__CustomImg-sc-1x0bfr6-1"]{content:"cdFmmG,"}/*!sc*/
.dqJGEV{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;height:10px;}/*!sc*/
data-styled.g66[id="LikeCounterstyle__Container-kp9n2u-0"]{content:"dqJGEV,"}/*!sc*/
.cEbOEy{margin-right:8px;}/*!sc*/
data-styled.g67[id="LikeCounterstyle__ButtonContainer-kp9n2u-1"]{content:"cEbOEy,"}/*!sc*/
.biZUop{padding:0;}/*!sc*/
data-styled.g68[id="LikeCounterstyle__CustomButton-kp9n2u-2"]{content:"biZUop,"}/*!sc*/
.hxtTFu{width:20px;height:20px;fill:#FFFFFF;stroke:#2A2A2A;stroke-width:2;}/*!sc*/
.hxtTFu:hover{fill:#e6005a;stroke:#e6005a;stroke-width:2;fill-opacity:'1';}/*!sc*/
data-styled.g69[id="LikeCounterstyle__CustomLikeIcon-kp9n2u-3"]{content:"hxtTFu,"}/*!sc*/
.hiqgTP{margin-bottom:15px;color:#2A2A2A;font-weight:normal;font-size:16px;font-family:'Roboto','Helvetica','Arial',sans-serif;line-height:22px;color:#a3a3a3;font-size:12px;}/*!sc*/
data-styled.g78[id="PostCardSignaturestyle__Container-zbxqg-0"]{content:"hiqgTP,"}/*!sc*/
.deFYH{padding-left:0;font-size:12px;}/*!sc*/
data-styled.g79[id="PostCardSignaturestyle__SignatureButton-zbxqg-1"]{content:"deFYH,"}/*!sc*/
.beynBk{fill:#a3a3a3;}/*!sc*/
data-styled.g80[id="PostCardSignaturestyle__CustomExpandIcon-zbxqg-2"]{content:"beynBk,"}/*!sc*/
.XqBiC{color:#e6005a;}/*!sc*/
data-styled.g110[id="BreadCrumbstyle__CustomLink-txx9sn-0"]{content:"XqBiC,"}/*!sc*/
.kIISjG{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-box-pack:center;-webkit-justify-content:center;-ms-flex-pack:center;justify-content:center;margin-bottom:15px;}/*!sc*/
.kIISjG >:not(:last-child){margin-right:10px;}/*!sc*/
@media only screen and (min-width:768px){.kIISjG{margin-bottom:0;}}/*!sc*/
data-styled.g111[id="ForumFooterstyle__SmallCtaContainer-yu3eug-0"]{content:"kIISjG,"}/*!sc*/
.crjKLc{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-flow:row wrap;-ms-flex-flow:row wrap;flex-flow:row wrap;-webkit-box-pack:center;-webkit-justify-content:center;-ms-flex-pack:center;justify-content:center;}/*!sc*/
.crjKLc >:not(:last-child){margin-right:10px;}/*!sc*/
data-styled.g112[id="Paginationstyle__RowContainer-d53ifi-0"]{content:"crjKLc,"}/*!sc*/
.fsDQEW{display:none;}/*!sc*/
data-styled.g113[id="Paginationstyle__Optional-d53ifi-1"]{content:"fsDQEW,"}/*!sc*/
.igTqKU{margin-top:15px;font-weight:600;-webkit-text-decoration:underline;text-decoration:underline;}/*!sc*/
.igTqKU:hover{-webkit-text-decoration:underline;text-decoration:underline;}/*!sc*/
data-styled.g114[id="Paginationstyle__SeeMoreButton-d53ifi-2"]{content:"igTqKU,"}/*!sc*/
.ioFwmU{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;margin-bottom:25px;}/*!sc*/
data-styled.g115[id="Paginationstyle__Container-d53ifi-3"]{content:"ioFwmU,"}/*!sc*/
.idnEnj{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:center;-webkit-justify-content:center;-ms-flex-pack:center;justify-content:center;min-width:35px;padding:7px 11px;color:#2A2A2A;background:#FFFFFF;border-radius:18px;}/*!sc*/
.idnEnj:hover{color:#e6005a;cursor:pointer;}/*!sc*/
.eRClxI{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:center;-webkit-justify-content:center;-ms-flex-pack:center;justify-content:center;min-width:35px;padding:7px 11px;color:#FFFFFF;background:#e6005a;border-radius:18px;}/*!sc*/
.eRClxI:hover{color:#FFFFFF;cursor:pointer;}/*!sc*/
data-styled.g116[id="PaginationItemstyle__Container-sc-1fdlb6e-0"]{content:"idnEnj,eRClxI,"}/*!sc*/
.cLorOg{-webkit-text-decoration:none;text-decoration:none;}/*!sc*/
data-styled.g117[id="PaginationItemstyle__LinkContent-sc-1fdlb6e-1"]{content:"cLorOg,"}/*!sc*/
.eCvIhI{color:inherit;-webkit-text-decoration:inherit;text-decoration:inherit;cursor:inherit;}/*!sc*/
data-styled.g118[id="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2"]{content:"eCvIhI,"}/*!sc*/
.AumzX{height:75px;padding-top:30px;}/*!sc*/
data-styled.g119[id="ActiveThreadstyle__LinkContent-sc-1pce28s-0"]{content:"AumzX,"}/*!sc*/
.cHsuON{display:block;}/*!sc*/
data-styled.g120[id="ActiveThreadstyle__CustomLink-sc-1pce28s-1"]{content:"cHsuON,"}/*!sc*/
.ezjMjR{margin-bottom:30px;padding:0 25px;border:2px solid #f39853;border-radius:13px;}/*!sc*/
data-styled.g121[id="ActiveThreadsstyle__CustomFieldset-sc-1as8dk-0"]{content:"ezjMjR,"}/*!sc*/
.bLwFTq{margin:0 auto;padding:10px;font-family:'Amatic SC','Roboto','Helvetica','Arial',sans-serif;}/*!sc*/
data-styled.g122[id="ActiveThreadsstyle__CustomLegend-sc-1as8dk-1"]{content:"bLwFTq,"}/*!sc*/
.Ikwdt{padding:0 15px;}/*!sc*/
.Ikwdt > *:not(:last-child){border-bottom:1px solid #d5d5d5;}/*!sc*/
data-styled.g123[id="ActiveThreadsstyle__Content-sc-1as8dk-2"]{content:"Ikwdt,"}/*!sc*/
.cSgIid{padding-top:10px;color:#2A2A2A;font-weight:normal;font-size:16px;font-family:'Roboto','Helvetica','Arial',sans-serif;line-height:22px;font-weight:600;font-size:16px;cursor:pointer;}/*!sc*/
data-styled.g188[id="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0"]{content:"cSgIid,"}/*!sc*/
.jkmxUM{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:justify;-webkit-justify-content:space-between;-ms-flex-pack:justify;justify-content:space-between;margin-bottom:15px;}/*!sc*/
.jkmxUM > *:not(:last-child){margin-right:20px;}/*!sc*/
data-styled.g189[id="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1"]{content:"jkmxUM,"}/*!sc*/
.btrLea{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:row;-ms-flex-direction:row;flex-direction:row;-webkit-box-flex:1;-webkit-flex-grow:1;-ms-flex-positive:1;flex-grow:1;}/*!sc*/
data-styled.g190[id="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2"]{content:"btrLea,"}/*!sc*/
.gSDPgE .tooltip{width:275px;margin-top:10px;margin-bottom:10px;padding:25px;color:#2A2A2A;background-color:#FFFFFF;border-radius:13px;box-shadow:0px 0px 4px 0px rgba(0,0,0,0.16);}/*!sc*/
.gSDPgE .arrow{left:130px !important;}/*!sc*/
.gSDPgE .arrow::before{color:#FFFFFF;box-shadow:0px 0px 4px 0px rgba(0,0,0,0.16);}/*!sc*/
data-styled.g191[id="DesktopPostCardHeaderstyle__CustomPopper-sc-17t45kd-3"]{content:"gSDPgE,"}/*!sc*/
.ewmuHh{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:row;-ms-flex-direction:row;flex-direction:row;}/*!sc*/
data-styled.g192[id="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4"]{content:"ewmuHh,"}/*!sc*/
.jTVLai{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;}/*!sc*/
data-styled.g193[id="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5"]{content:"jTVLai,"}/*!sc*/
.iVvXSp{margin-right:10px;}/*!sc*/
data-styled.g194[id="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6"]{content:"iVvXSp,"}/*!sc*/
.cSgZVd{color:#2A2A2A;font-weight:normal;font-size:16px;font-family:'Roboto','Helvetica','Arial',sans-serif;line-height:22px;color:#a3a3a3;font-weight:100;font-size:12px;}/*!sc*/
data-styled.g195[id="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7"]{content:"cSgZVd,"}/*!sc*/
.cngpQz{margin:auto;text-align:right;}/*!sc*/
data-styled.g196[id="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8"]{content:"cngpQz,"}/*!sc*/
.hxFQiP{width:20px;height:20px;}/*!sc*/
data-styled.g199[id="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11"]{content:"hxFQiP,"}/*!sc*/
.fXVIbQ{padding:10px;}/*!sc*/
data-styled.g200[id="DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12"]{content:"fXVIbQ,"}/*!sc*/
.rGdlb{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-flow:column nowrap;-ms-flex-flow:column nowrap;flex-flow:column nowrap;}/*!sc*/
data-styled.g203[id="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0"]{content:"rGdlb,"}/*!sc*/
.eurmgj{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:row;-ms-flex-direction:row;flex-direction:row;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:justify;-webkit-justify-content:space-between;-ms-flex-pack:justify;justify-content:space-between;height:30px;}/*!sc*/
data-styled.g204[id="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1"]{content:"eurmgj,"}/*!sc*/
.gIugAn{position:relative;margin-bottom:32px;overflow:visible;font-size:14px;}/*!sc*/
.gIugAn::before{position:absolute;top:0;left:0;width:0%;height:0%;background:linear-gradient(rgba(255,255,255,0) 250px,white);content:'';}/*!sc*/
.gIugAn ul{list-style-position:inside;list-style-type:disc;}/*!sc*/
.gIugAn ol{list-style-position:inside;list-style-type:decimal;}/*!sc*/
.gIugAn b{font-weight:600;}/*!sc*/
.gIugAn i{font-style:italic;}/*!sc*/
.gIugAn a{color:#2A2A2A;}/*!sc*/
.gIugAn img{max-width:100%;max-height:300px;}/*!sc*/
.gIugAn .CF_bbcode_img{cursor:pointer;}/*!sc*/
.gIugAn .quote{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;}/*!sc*/
.gIugAn .quote br{display:none;}/*!sc*/
.gIugAn .container br{display:none;}/*!sc*/
data-styled.g208[id="DesktopPostCardstyle__ThreadContent-t45zrm-3"]{content:"gIugAn,"}/*!sc*/
.btHWAz{display:block;width:120px;-webkit-transition:250ms;transition:250ms;opacity:0;}/*!sc*/
data-styled.g209[id="DesktopPostCardstyle__ReplyButton-t45zrm-4"]{content:"btHWAz,"}/*!sc*/
.iiMZYv{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;-webkit-box-pack:justify;-webkit-justify-content:space-between;-ms-flex-pack:justify;justify-content:space-between;margin-bottom:30px;padding:30px 30px;background-color:#FFFFFF;box-shadow:2px 4px 8px 1px rgba(164,164,164,0.13);max-height:none;border-radius:13px;}/*!sc*/
.iiMZYv:hover .DesktopPostCardstyle__ReplyButton-t45zrm-4{opacity:1;}/*!sc*/
data-styled.g210[id="DesktopPostCardstyle__PostContainer-t45zrm-5"]{content:"iiMZYv,"}/*!sc*/
.ddwMKL{max-height:none;margin-bottom:30px;position:relative;z-index:2;display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;padding:15px 25px;background-color:#F6F7F1;border-radius:8px;}/*!sc*/
data-styled.g211[id="DesktopQuotedPostCardstyle__Container-sc-1opvjiu-0"]{content:"ddwMKL,"}/*!sc*/
.fmZwVl{overflow:visible;}/*!sc*/
data-styled.g212[id="DesktopQuotedPostCardstyle__ContentContainer-sc-1opvjiu-1"]{content:"fmZwVl,"}/*!sc*/
.iSqKAc{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:row;-ms-flex-direction:row;flex-direction:row;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;margin-bottom:10px;}/*!sc*/
data-styled.g213[id="DesktopQuotedPostCardstyle__Header-sc-1opvjiu-2"]{content:"iSqKAc,"}/*!sc*/
.jroQyO{width:15px;height:15px;margin-right:5px;}/*!sc*/
data-styled.g214[id="DesktopQuotedPostCardstyle__CustomReplyIcon-sc-1opvjiu-3"]{content:"jroQyO,"}/*!sc*/
.chMnry{margin-right:5px;}/*!sc*/
data-styled.g215[id="DesktopQuotedPostCardstyle__Title-sc-1opvjiu-4"]{content:"chMnry,"}/*!sc*/
.iVeKOx{color:#e6005a;font-weight:600;}/*!sc*/
.iVeKOx:hover{cursor:pointer;}/*!sc*/
data-styled.g216[id="DesktopQuotedPostCardstyle__Author-sc-1opvjiu-5"]{content:"iVeKOx,"}/*!sc*/
.ljEzlN .tooltip{width:277px;margin-top:10px;margin-bottom:10px;padding:24px;color:#2A2A2A;background-color:#FFFFFF;border-radius:13px;box-shadow:0px 0px 4px 0px rgba(0,0,0,0.16);}/*!sc*/
.ljEzlN .arrow{left:130px !important;}/*!sc*/
.ljEzlN .arrow::before{color:#FFFFFF;box-shadow:0px 0px 4px 0px rgba(0,0,0,0.16);}/*!sc*/
data-styled.g217[id="DesktopQuotedPostCardstyle__CustomPopper-sc-1opvjiu-6"]{content:"ljEzlN,"}/*!sc*/
.jBKeFF{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:row;-ms-flex-direction:row;flex-direction:row;}/*!sc*/
data-styled.g218[id="DesktopQuotedPostCardstyle__TooltipZone-sc-1opvjiu-7"]{content:"jBKeFF,"}/*!sc*/
.gGtCJT{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;padding:25px 55px;background-color:#F6F7F1;border-top-left-radius:13px;border-top-right-radius:13px;box-shadow:2px 4px 8px 1px rgba(164,164,164,0.13);}/*!sc*/
data-styled.g223[id="ThreadDesktopHeaderstyle__Container-jm5nf5-0"]{content:"gGtCJT,"}/*!sc*/
.cHjCZH{font-weight:600;font-size:20px;font-family:'Roboto','Helvetica','Arial',sans-serif;margin-bottom:15px;color:#e6005a;text-align:center;}/*!sc*/
data-styled.g224[id="ThreadDesktopHeaderstyle__ThreadTitle-jm5nf5-1"]{content:"cHjCZH,"}/*!sc*/
.drGwis{color:#2A2A2A;font-weight:normal;font-size:16px;font-family:'Roboto','Helvetica','Arial',sans-serif;line-height:22px;color:#939393;font-weight:100;font-size:12px;text-align:center;}/*!sc*/
data-styled.g225[id="ThreadDesktopHeaderstyle__ThreadLastPostDate-jm5nf5-2"]{content:"drGwis,"}/*!sc*/
.eEiLbs{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:center;-webkit-justify-content:center;-ms-flex-pack:center;justify-content:center;}/*!sc*/
data-styled.g226[id="ThreadDesktopHeaderstyle__ThreadSubtitleContainer-jm5nf5-3"]{content:"eEiLbs,"}/*!sc*/
.cJUNkI{margin-right:3px;color:#2A2A2A;}/*!sc*/
data-styled.g227[id="ThreadDesktopHeaderstyle__ThreadPostsCount-jm5nf5-4"]{content:"cJUNkI,"}/*!sc*/
.igLkWh{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-flow:column nowrap;-ms-flex-flow:column nowrap;flex-flow:column nowrap;padding:0;}/*!sc*/
data-styled.g228[id="ThreadDesktopstyle__Container-sc-2f61v0-0"]{content:"igLkWh,"}/*!sc*/
.fzhhcj{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-flow:row nowrap;-ms-flex-flow:row nowrap;flex-flow:row nowrap;-webkit-align-items:flex-start;-webkit-box-align:flex-start;-ms-flex-align:flex-start;align-items:flex-start;-webkit-box-pack:start;-webkit-justify-content:flex-start;-ms-flex-pack:start;justify-content:flex-start;margin-bottom:25px;padding-right:25px;background-color:#FFFFFF;}/*!sc*/
data-styled.g229[id="ThreadDesktopstyle__ThreadPageContainer-sc-2f61v0-1"]{content:"fzhhcj,"}/*!sc*/
.fZemwZ{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;--margin:25px;width:calc(1000px - 300px - 3 * var(--margin));margin:0 var(--margin);}/*!sc*/
data-styled.g230[id="ThreadDesktopstyle__ThreadContainer-sc-2f61v0-2"]{content:"fZemwZ,"}/*!sc*/
.bFUkyW{-webkit-flex-shrink:0;-ms-flex-negative:0;flex-shrink:0;min-width:300px;}/*!sc*/
data-styled.g231[id="ThreadDesktopstyle__Banner300AtfContainer-sc-2f61v0-3"]{content:"bFUkyW,"}/*!sc*/
.lbvJUs{margin-bottom:25px;}/*!sc*/
data-styled.g232[id="ThreadDesktopstyle__BreadCrumbContainer-sc-2f61v0-4"]{content:"lbvJUs,"}/*!sc*/
.bzTFtD{width:100%;}/*!sc*/
data-styled.g233[id="ThreadDesktopstyle__PostsContainer-sc-2f61v0-5"]{content:"bzTFtD,"}/*!sc*/
</style></head><body><div id="__next"><div class="Rootstyle__RootContainer-sc-1faqzk7-0 cpmnmV"><main class="Rootstyle__PageContent-sc-1faqzk7-1 igzlTH"><div data-ad-skin-insert-after="true" data-ad-sticky-follow="true"><div><script>window.__REACT_ESI__ = window.__REACT_ESI__ || {}; window.__REACT_ESI__['NetmumsHeader'] = {"htmlData":"\u003c!-- Netmums - Start - BEFORE -->\u003cheader class=\"nmg-header\" data-menu=\"/esi/nav\" data-loaded=\"false\">\u003cdiv class=\"nmg-header__top\">\u003cdiv class=\"nmg-header__top__left\">\u003ca href=\"https://www.netmums.com/search\" title=\"Search\" class=\"nmg-header__btn nmg-header__btn--search\" aria-label=\"search\" aria-expanded=\"false\" aria-controls=\"search\" id=\"searchButton\" onclick=\"af_ua_trackEvent('Nav', 'Search')\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--search\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconSearch\" />\u003c/svg>\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--cross\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconCross\" />\u003c/svg>\u003cspan class=\"nmg-header__search__label nmg-hidden-mobile\">Search\u003c/span>\u003c/a>\u003cdiv class=\"nmg-header__search\" id=\"search\" aria-labelledby=\"searchButton\">\u003cform action=\"https://www.netmums.com/search\" method=\"post\">\u003cinput name=\"data[Page][keywords]\" type=\"text\" placeholder=\"Search…\" class=\"nmg-header__search__field\" autocomplete=\"off\" aria-label=\"Search\" id=\"PageKeywords\">\u003c/form>\u003c/div>\u003c/div>\u003c!-- .nmg-header__top__left -->\u003cdiv class=\"nmg-header__brand\">\u003ca href=\"https://www.netmums.com/\" title=\"Netmums Homepage\" onclick=\"af_ua_trackEvent('Nav', 'Logo')\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--logo-text nmg-hidden-mobile\" width=\"136\" height=\"30\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconLogoText\">\u003c/use>\u003c/svg>\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--logo-circle nmg-hidden-desktop\" width=\"40\" height=\"40\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconLogoCircle\" />\u003c/svg>\u003c/a>\u003c/div>\u003c!-- .nmg-header__brand -->\u003cdiv class=\"nmg-header__top__right\">\u003ca href=\"https://www.netmums.com/subscribe\" class=\"nmg-header__btn nmg-header__btn--newsletters nmg-hidden-mobile\" onclick=\"af_ua_trackEvent('Nav', 'Newsletters')\">\u003cspan>Newsletters\u003c/span>\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--send\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconSend\" />\u003c/svg>\u003c/a>\u003cbutton class=\"nmg-header__menu-btn nmg-hidden-desktop\" aria-label=\"menu\" aria-expanded=\"false\" aria-controls=\"menu\" id=\"menuButton\" onclick=\"af_ua_trackEvent('Nav', 'Menu')\">\u003cspan>\u003c/span>\u003c/button>\u003c/div>\u003c!-- .nmg-header__top__right -->\u003c/div>\u003c!-- .nmg-header__top -->\u003cdiv class=\"nmg-header__secondary\">\u003cdiv class=\"nmg-header__menu\" id=\"menu\" aria-labelledby=\"menuButton\">\u003cnav class=\"nmg-nav\">\u003cul class=\"nmg-nav__list\">\u003cli class=\"nmg-nav__item nmg-hidden-desktop\">\u003ca href=\"https://www.netmums.com/\" class=\"nmg-nav__link\">Home\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/coffeehouse/\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navChat\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Chat\u003c/a>\u003cul class=\"nmg-nav__list-secondary\" id=\"navChat\">\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/coffeehouse/\" class=\"nmg-nav__link-secondary\">Chat homepage\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/info/netmums-drop-in-clinic-parent-supporter-service\" class=\"nmg-nav__link-secondary\">Netmums drop-in clinic\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/coffeehouse/other-chat-514/general-chat-18/\" class=\"nmg-nav__link-secondary\">General chat\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/coffeehouse/becoming-mum-ttc-64/trying-conceive-clubs-525/\" class=\"nmg-nav__link-secondary\">Join your TTC club\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/coffeehouse/becoming-mum-ttc-64/am-i-pregnant-874/\" class=\"nmg-nav__link-secondary\">Am I pregnant?\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/coffeehouse/becoming-mum-ttc-64/early-pregnancy-signs-symptoms-537/\" class=\"nmg-nav__link-secondary\">Early pregnancy symptoms\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/coffeehouse/becoming-mum-pregnancy-996/due-dates-clubs-189/\" class=\"nmg-nav__link-secondary\">Join your due date club\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/coffeehouse/being-mum-794/premature-scbu-babies-564/\" class=\"nmg-nav__link-secondary\">Premature and SCBU babies\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/coffeehouse/drop-clinic-984/baby-child-health-989/\" class=\"nmg-nav__link-secondary\">Baby and child health\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/coffeehouse/being-mum-794/toddlers-1-3-years-59/\" class=\"nmg-nav__link-secondary\">Toddlers (1-3 years)\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/coffeehouse/being-mum-794/children-4-11-years-60/\" class=\"nmg-nav__link-secondary\">Children (4-11 years)\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/coffeehouse/being-mum-794/tweens-teens-61/\" class=\"nmg-nav__link-secondary\">Tweens and teens\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item-secondary\">\u003ca href=\"https://www.netmums.com/coffeehouse/life-504/family-other-relationships-50/\" class=\"nmg-nav__link-secondary\">Family and other relationships\u003c/a>\u003c/li>\u003c/ul>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/pregnancy/trying-for-a-baby\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navTryingforABaby\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Trying for a baby\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/pregnancy\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navPregnancy\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Pregnancy\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/baby\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navBaby\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Baby\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/parenting\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navParenting\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Parenting\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/local\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navLocal\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Local\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/activities\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navActivities\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Activities\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/reviews/\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navReviews\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Reviews\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/valentines-day\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navSeasonal\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Valentine's day\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/life\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navLife\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Life\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/recipes\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navRecipes\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Recipes\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/pregnancy/baby-names\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navBabynames\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Baby names\u003c/a>\u003c/li>\u003cli class=\"nmg-nav__item nmg-nav__item--children\">\u003ca href=\"https://www.netmums.com/coronavirus\" class=\"nmg-nav__link\" aria-expanded=\"false\" aria-controls=\"navCoronavirus\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-down\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown\" />\u003c/svg>Coronavirus\u003c/a>\u003c/li>\u003c/ul>\u003cbutton class=\"nmg-nav__toggle nmg-hidden-mobile\" aria-hidden=\"true\" aria-controls=\"menu\">\u003csvg aria-hidden=\"true\" class=\"nmg-icon nmg-icon--chevron-right\" width=\"24\" height=\"24\">\u003cuse xlink:href=\"https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronRight\" />\u003c/svg>\u003c/button>\u003c/nav>\u003c!-- .nmg-nav -->\u003c/div>\u003c!-- .nmg-header__menu -->\u003c/div>\u003c!-- .nmg-header__secondary -->\u003c/header>\u003c!-- Netmums - End - BEFORE -->","serverSideRendered":true};document.currentScript.remove();</script><div><!-- Netmums - Start - BEFORE --><header class="nmg-header" data-loaded="false" data-menu="/esi/nav"><div class="nmg-header__top"><div class="nmg-header__top__left"><a aria-controls="search" aria-expanded="false" aria-label="search" class="nmg-header__btn nmg-header__btn--search" href="https://www.netmums.com/search" id="searchButton" onclick="af_ua_trackEvent('Nav', 'Search')" title="Search"><svg aria-hidden="true" class="nmg-icon nmg-icon--search" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconSearch"></use></svg><svg aria-hidden="true" class="nmg-icon nmg-icon--cross" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconCross"></use></svg><span class="nmg-header__search__label nmg-hidden-mobile">Search</span></a><div aria-labelledby="searchButton" class="nmg-header__search" id="search"><form action="https://www.netmums.com/search" method="post"><input aria-label="Search" autocomplete="off" class="nmg-header__search__field" id="PageKeywords" name="data[Page][keywords]" placeholder="Search…" type="text"/></form></div></div><!-- .nmg-header__top__left --><div class="nmg-header__brand"><a href="https://www.netmums.com/" onclick="af_ua_trackEvent('Nav', 'Logo')" title="Netmums Homepage"><svg aria-hidden="true" class="nmg-icon nmg-icon--logo-text nmg-hidden-mobile" height="30" width="136"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconLogoText"></use></svg><svg aria-hidden="true" class="nmg-icon nmg-icon--logo-circle nmg-hidden-desktop" height="40" width="40"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconLogoCircle"></use></svg></a></div><!-- .nmg-header__brand --><div class="nmg-header__top__right"><a class="nmg-header__btn nmg-header__btn--newsletters nmg-hidden-mobile" href="https://www.netmums.com/subscribe" onclick="af_ua_trackEvent('Nav', 'Newsletters')"><span>Newsletters</span><svg aria-hidden="true" class="nmg-icon nmg-icon--send" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconSend"></use></svg></a><button aria-controls="menu" aria-expanded="false" aria-label="menu" class="nmg-header__menu-btn nmg-hidden-desktop" id="menuButton" onclick="af_ua_trackEvent('Nav', 'Menu')"><span></span></button></div><!-- .nmg-header__top__right --></div><!-- .nmg-header__top --><div class="nmg-header__secondary"><div aria-labelledby="menuButton" class="nmg-header__menu" id="menu"><nav class="nmg-nav"><ul class="nmg-nav__list"><li class="nmg-nav__item nmg-hidden-desktop"><a class="nmg-nav__link" href="https://www.netmums.com/">Home</a></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navChat" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/coffeehouse/"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Chat</a><ul class="nmg-nav__list-secondary" id="navChat"><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/coffeehouse/">Chat homepage</a></li><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/info/netmums-drop-in-clinic-parent-supporter-service">Netmums drop-in clinic</a></li><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/coffeehouse/other-chat-514/general-chat-18/">General chat</a></li><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/coffeehouse/becoming-mum-ttc-64/trying-conceive-clubs-525/">Join your TTC club</a></li><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/coffeehouse/becoming-mum-ttc-64/am-i-pregnant-874/">Am I pregnant?</a></li><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/coffeehouse/becoming-mum-ttc-64/early-pregnancy-signs-symptoms-537/">Early pregnancy symptoms</a></li><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/coffeehouse/becoming-mum-pregnancy-996/due-dates-clubs-189/">Join your due date club</a></li><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/coffeehouse/being-mum-794/premature-scbu-babies-564/">Premature and SCBU babies</a></li><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/coffeehouse/drop-clinic-984/baby-child-health-989/">Baby and child health</a></li><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/coffeehouse/being-mum-794/toddlers-1-3-years-59/">Toddlers (1-3 years)</a></li><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/coffeehouse/being-mum-794/children-4-11-years-60/">Children (4-11 years)</a></li><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/coffeehouse/being-mum-794/tweens-teens-61/">Tweens and teens</a></li><li class="nmg-nav__item-secondary"><a class="nmg-nav__link-secondary" href="https://www.netmums.com/coffeehouse/life-504/family-other-relationships-50/">Family and other relationships</a></li></ul></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navTryingforABaby" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/pregnancy/trying-for-a-baby"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Trying for a baby</a></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navPregnancy" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/pregnancy"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Pregnancy</a></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navBaby" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/baby"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Baby</a></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navParenting" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/parenting"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Parenting</a></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navLocal" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/local"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Local</a></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navActivities" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/activities"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Activities</a></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navReviews" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/reviews/"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Reviews</a></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navSeasonal" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/valentines-day"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Valentine's day</a></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navLife" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/life"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Life</a></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navRecipes" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/recipes"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Recipes</a></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navBabynames" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/pregnancy/baby-names"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Baby names</a></li><li class="nmg-nav__item nmg-nav__item--children"><a aria-controls="navCoronavirus" aria-expanded="false" class="nmg-nav__link" href="https://www.netmums.com/coronavirus"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-down" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronDown"></use></svg>Coronavirus</a></li></ul><button aria-controls="menu" aria-hidden="true" class="nmg-nav__toggle nmg-hidden-mobile"><svg aria-hidden="true" class="nmg-icon nmg-icon--chevron-right" height="24" width="24"><use xlink:href="https://www.netmums.com/assets/nmg/img/nmg-icon/nmg-icon.svg#nmgIconChevronRight"></use></svg></button></nav><!-- .nmg-nav --></div><!-- .nmg-header__menu --></div><!-- .nmg-header__secondary --></header><!-- Netmums - End - BEFORE --></div></div></div><div><div class="ThreadDesktopstyle__Container-sc-2f61v0-0 igLkWh"><div class="BannerAtfstyle__BannerAtfContainer-sc-1eqlkbi-0 eFYlhl"><div data-ad="banner_atf"></div></div><div class="ThreadDesktopstyle__ThreadPageContainer-sc-2f61v0-1 fzhhcj" data-ad-skin-main="true"><div class="ThreadDesktopstyle__ThreadContainer-sc-2f61v0-2 fZemwZ" data-ad-sticky-reference="#pegasus_300_atf"><div class="ThreadDesktopstyle__BreadCrumbContainer-sc-2f61v0-4 lbvJUs"><nav aria-label="Breadcrumb" class="MuiTypography-root MuiBreadcrumbs-root MuiTypography-body1 MuiTypography-colorTextSecondary"><ol class="MuiBreadcrumbs-ol"><li class="MuiBreadcrumbs-li"><a class="MuiTypography-root MuiLink-root MuiLink-underlineHover BreadCrumbstyle__CustomLink-txx9sn-0 XqBiC MuiTypography-colorPrimary" href="/coffeehouse">Netmums Forum</a></li><li aria-hidden="true" class="MuiBreadcrumbs-separator">/</li><li class="MuiBreadcrumbs-li"><a class="MuiTypography-root MuiLink-root MuiLink-underlineHover BreadCrumbstyle__CustomLink-txx9sn-0 XqBiC MuiTypography-colorPrimary" href="https://www.netmums.com/coffeehouse/money-advice-1127/">Money advice</a></li><li aria-hidden="true" class="MuiBreadcrumbs-separator">/</li><li class="MuiBreadcrumbs-li"><a class="MuiTypography-root MuiLink-root MuiLink-underlineHover BreadCrumbstyle__CustomLink-txx9sn-0 XqBiC MuiTypography-colorPrimary" href="https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/">Budgeting</a></li></ol></nav></div><div class="ThreadDesktopHeaderstyle__Container-jm5nf5-0 gGtCJT"><h1 class="ThreadDesktopHeaderstyle__ThreadTitle-jm5nf5-1 cHjCZH">To all you lovely savvy shoppers</h1><div class="ThreadDesktopHeaderstyle__ThreadSubtitleContainer-jm5nf5-3 eEiLbs"><p class="ThreadDesktopHeaderstyle__ThreadLastPostDate-jm5nf5-2 ThreadDesktopHeaderstyle__ThreadPostsCount-jm5nf5-4 drGwis cJUNkI">14300 answers<!-- --> /</p><p class="ThreadDesktopHeaderstyle__ThreadLastPostDate-jm5nf5-2 drGwis">Last post: 11/9/2019 at 11:55 AM</p></div></div><div class="ThreadDesktopstyle__PostsContainer-sc-2f61v0-5 bzTFtD"><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8905360"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo MuiAvatar-colorDefault"><img class="Avatarstyle__CustomImg-sc-1x0bfr6-1 cdFmmG" src="/coffeehouse/images/_next/avatar-default.png"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">Anonymous</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 10:25 AM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div><img alt=":hiya:" class="wysiwyg_smiley" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/greet024.gif" title=":hiya:"/> Morning ladies. <br/><br/>Me and Ryan slept downstairs last night as OH has a chest infection and was coughing and spluttering all night. <br/><br/>Andrea hope you're okay xx<br/><br/>Claire I found when I cut fizzy drinks out I also stopped with so many headaches. We're in the area that starts the universal credits in April. Apparently it will make families no worse off, it's just compiling payments to monthly. This is what we have been told anyway that we're not going to be worse off. Hopefully anyway! <br/><br/>Cheryl I know how you feel with a bad chest. I only have part function of my right lung after having 8 blood clots on my lung and pneumonia at the same time ever since I have not been right and that was November 2009. Nothing worse than a sore chest. Have you tried sleeping sat upright? I found this helped and for 8 months after getting ill I slept downstairs sat up in a chair. <br/><br/>Not got much planned today. Need to clean up and tidy round. OH has got football tonight, even though he is ill. They're playing reading so he's looking forward to that.<br/><br/>DS1's knee is a lot better this morning but looks horrendous when I changed the dressing <img alt=":yuk:" class="wysiwyg_smiley" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/yuk.gif" title=":yuk:"/><br/><br/>We have lovely blue sky today and it's warmish out but with a fresh feeling to the air. Sold more clothes on ebay last night, don't have to deal with postage as she is collecting them. Gutted they didn't go for more £4.20 for £130 worth of clothes! Some are brand new next too.</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="SeeLastAnswerCtastyle__Container-sc-1lpozhn-0 gOHhMk"><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined SeeLastAnswerCtastyle__SeeLastAnswerButton-sc-1lpozhn-1 kiAQko MuiButton-outlinedSizeSmall MuiButton-sizeSmall MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">See last answer</span></button></div><div class="InterPostComponentstyle__MarginableContainer-e122kk-1 bLGqZf"><div class="Videostyle__Container-jonus6-0 eKVeqz"><div class="Videostyle__VideoContainer-jonus6-1 ccJNiz"><div><script>window.__REACT_ESI__ = window.__REACT_ESI__ || {}; window.__REACT_ESI__['VideoPlayer'] = {"startOnSight":"1","device":"desktop","videoId":538148,"sticky":true,"htmlData":"\u003c!-- ESI - VideoPlayer : http://www.netmums.com/reloaded/esi/videoplayer.php?device=desktop&position=MTF_seealso&startOnSight=1&stickTo=bottom-right&videoid=538148 -->\n\n\u003cscript type=\"text/javascript\" crossorigin=\"anonymous\" src=\"https://static.afcdn.com/relnm/concat.js?PCM-lVLBbsIwDP0ajqsok7jtgCZtmsQBtolr5bamBNI4ipOh_v0cqkKLEBQpSl6d9_oc2xs0JbnJ7EOZHTrlwRQoX6xqq_HrEkv2_PZOtSWDxgvhT5VIKw0NRrE9ARaUQQnWoxvL3x9bHPk77y1PXhcSllWUJjnfFlRLSKvcgVMYhc36W6vl2t83gnL1ZGp9RT-5TVcoSYULctiDGXuHUDMaxqRW5r6Dd1AcWoPuB5tI-I1xZarR6s_FEzqrQ6VMlJWYh6o7x4rYq-LQnMFYGQRPsYY9OFpqQDdiNsCDTkDYyl7BraCmHHQm9a3JZGI84GgqgXdtCSg8nO7glebRXZEETm1ZUlVdjY4NOYdcQJrMk-k58MLSPY2ezICd05HxArrBunopbG8YdVc_yKzowTyylwm8T6mlAa1HMpvOUllpKsc_\">\u003c/script>\n\u003cscript type=\"text/javascript\">\n window.afSession = window.afSession || new AufSession();\n \u003c/script>\n\u003cscript type=\"text/javascript\">\n window.afLogger = window.afLogger || new Aflog.Logger({\n context: {\n revision: '097f376dfff2902df245542638f65d9f32d54955'\n }\n });\n af_addEvent(window, 'beforeunload', function() {\n afLogger.sendLogsToBigQuery();\n }, false);\n \u003c/script>\n\u003cscript type=\"text/javascript\">\n if (Math.random() \u003c 0.1) {\n (function() {\n function hsLogError(message, category, sampling) {\n var d = message.split('').reverse().join('');\n new Image().src = 'https://aufeminin.netmums.com/reloaded/errpix.php?'\n + 'd=' + encodeURIComponent(d)\n + (sampling ? '&s=' + sampling : '')\n + (category ? '&c=' + encodeURIComponent(category) : '');\n if (window.console && console.error) console.error('JS Catchall: ' + message);\n }\n\n var af_JsElementBlocked = false;\n var af_nbErrorsToReport = 10;\n window.onerror = function(message, url, line, column, error) {\n if (af_nbErrorsToReport-- \u003c 0) return;\n\n var category = 'JS - window.onerror';\n\n if (typeof message === 'string') {\n if (message.indexOf('Script error') !== -1) {\n category += ' - external';\n }\n\n if (url && url.indexOf(document.domain) === -1) {\n message += ' @ ' + url;\n if (typeof line !== undefined) {\n message += ':' + line;\n }\n if (typeof column !== undefined) {\n message += ':' + column;\n }\n }\n } else if (typeof message == 'object' && message.srcElement && message.target) {\n if (message.srcElement == '[object HTMLScriptElement]' && message.target == '[object HTMLScriptElement]') {\n message = 'Error loading script';\n } else {\n message = 'Event Error - target:' + message.target + ' srcElement:' + message.srcElement;\n af_JsElementBlocked = true;\n }\n } else if (typeof message == 'object') {\n message = 'MESSAGE_OBJECT: ' + message.constructor.name;\n } else {\n message = 'MESSAGE_NOSTRING: ' + typeof message;\n }\n\n hsLogError(message, category);\n\n return true;\n };\n\n var af_taggingToReport = true;\n function checkTagging() {\n if (!af_taggingToReport) return;\n af_taggingToReport = false;\n if (window.top != window.self) return; // Iframe\n if (af_JsElementBlocked) return; // Errors when loading page\n\n if (!window.af_dataLayer && window.location.href.indexOf('doctissimo.fr') === -1) {\n hsLogError('GTM data layer missing', 'GTM', 1);\n }\n if (!window.af_ua_trackEvent) {\n hsLogError('UA track event function missing', 'GTM', 1);\n }\n }\n\n if (window.addEventListener) {\n window.addEventListener('load', checkTagging, false);\n } else if (window.attachEvent) {\n window.attachEvent('onload', checkTagging);\n }\n })();\n }\u003c/script>\n\u003clink href=\"https&#x3A;&#x2F;&#x2F;cdn.jwplayer.com\" rel=\"preconnect\">\n\u003clink href=\"https&#x3A;&#x2F;&#x2F;aufeminin.netmums.com\" rel=\"preconnect\">\n\u003clink href=\"https&#x3A;&#x2F;&#x2F;static.afcdn.com\" rel=\"preconnect\">\n\u003clink href=\"https&#x3A;&#x2F;&#x2F;assets.afcdn.com\" rel=\"preconnect\">\n\u003clink href=\"https://static.afcdn.com/relnm/concat.css?PCM-c87PLcjPS80r0S_LTEnND8hJrEwtQmbrFScXF-sZGRgZApGhIZACAA\" media=\"all\" rel=\"stylesheet\" type=\"text/css\">\n\n\u003cdiv id=\"af_player_60253f243e7451_42269561-container\" class=\"af-videoplayer\">\n\t\u003cvideo id=\"af_player_60253f243e7451_42269561\"\n\t\t   class=\" \"\n\t\t   poster=\"https://assets.afcdn.com/video35/20180116/v538148_a-message-to-my-birth-mother_w720h720c1.jpg\"\n\t\t   preload=\"none\"\n\t\t   controls\n\t\t   playsinline\n\t\tmuted\t>\n\t\t\t\t\t\u003csource src=\"https://assets.afcdn.com/video35/20180116/v538148_a-message-to-my-birth-mother.m3u8\" type=\"application/x-mpegURL\">\n\t\t\t\t\t\t\u003csource src=\"https://assets.afcdn.com/video35/20180116/v538148_a-message-to-my-birth-mother_HD.mp4\" type=\"video/mp4\">\n\t\t\t\t\t\t\u003csource src=\"https://assets.afcdn.com/video35/20180116/v538148_a-message-to-my-birth-mother_sd.mp4\" type=\"video/mp4\">\n\t\t\t\t\t\u003cp class=\"vjs-no-js\">\n\t\t\tYour browser cannot play this video.\t\t\u003c/p>\n\t\u003c/video>\n\u003c/div>\n\u003cscript>\n\t(function() {\n\t\t\t\t// JS logger\n\t\tvar logger = new VideoLogger({\n\t\t\tcontext: {\n\t\t\t\tadPlayer: {\n\t\t\t\t\tname: 'JWPlayer'\n\t\t\t\t},\n\t\t\t\tplayer: {\n\t\t\t\t\tname: 'JWPlayer',\n\t\t\t\t\tversion: 'JWP ' + jwplayer.version.substr(0,jwplayer.version.indexOf('+')),\n\t\t\t\t\tvideoId: 538148,\n\t\t\t\t\tcontext: 'ESI',\n\t\t\t\t\tposition: 'MTF_seealso',\n\t\t\t\t}\n\t\t\t}\n\t\t});\n\t\tvar gaTracker = AufVideo.Tracker.GA.get();\n\t\t\t\tAufVideo.Tracker.Comscore.init();\n\t\t\n\t\t\t\t\t\tvar player = new AufVideo.JWPlayer({\n\t\t\t\t\tid: 'af_player_60253f243e7451_42269561',\n\t\t\t\t\taspectRatio: \"16:9\",\n\t\t\t\t\tmuted: true,\n\t\t\t\t\tvideoId: 538148,\n\t\t\t\t\ttitle: \"A message to my birth mother\",\n\t\t\t\t\theight: 720,\n\t\t\t\t\tplaylist: {\n\t\t\t\t\t\tsources: [{\"file\":\"https:\\/\\/assets.afcdn.com\\/video35\\/20180116\\/v538148_a-message-to-my-birth-mother.m3u8\",\"type\":\"application\\/x-mpegURL\"},{\"file\":\"https:\\/\\/assets.afcdn.com\\/video35\\/20180116\\/v538148_a-message-to-my-birth-mother_HD.mp4\",\"type\":\"video\\/mp4\",\"label\":\"HD\"},{\"file\":\"https:\\/\\/assets.afcdn.com\\/video35\\/20180116\\/v538148_a-message-to-my-birth-mother_sd.mp4\",\"type\":\"video\\/mp4\",\"label\":\"SD\"}],\n\t\t\t\t\t\timage: \"https:\\/\\/assets.afcdn.com\\/video35\\/20180116\\/v538148_a-message-to-my-birth-mother_w720h720c1.jpg\"\t\t\t\t\t},\n\t\t\t\t});\n\t\t\t\t\t\tplayer.publish('video.player.setup', { file: 'https://assets.afcdn.com/video35/20180116/v538148_a-message-to-my-birth-mother.m3u8' });\n\t\t\n\t\tvar adPlayer;\n\t\t\t\t\twindow.unfyVideoPlacement =  [];\n\t\t\twindow.unfyVideoPlacement.push(\n\t\t\t\t{\n\t\t\t\t\tvideoId: 538148,\n\t\t\t\t\tcontext: 'ESI',\n\t\t\t\t\tadUnit: '/229683329/netmums/video/preroll_MTF_seealso',\n\t\t\t\t\tplacementId: 'preroll_mtf_seealso',\n\t\t\t\t\tplayerName: 'jwplayer',\n\t\t\t\t\tadPlayerName: 'jwplayer-ads',\n\t\t\t\t\tplayerId: 'af_player_60253f243e7451_42269561'\n\t\t\t});\n\n\t\t\t// Ad player configuration\n\t\t\tvar adTargeting = ';'\n\t\t\t\t+ ';player=jwplayer'\n\t\t\t\t+ ';adplayer=jwplayer-ads'\n\t\t\t\t+ ';context=ESI'\n\t\t\t\t+ ';video=538148v';\n\n\t\t\t\t\t\t\t\tvar adPlayer = new AufVideo.JWPlayerAds({\n\t\t\t\t\t\tid: 'af_player_60253f243e7451_42269561',\n\t\t\t\t\t\tplayer: player,\n\t\t\t\t\t\tadUnit: '/229683329/netmums/video/preroll_MTF_seealso',\n\t\t\t\t\t\tadTargeting: adTargeting,\n\t\t\t\t\t\tallowMultipleTinyAds: true,\n\t\t\t\t\t\tsiteId: 35,\n\t\t\t\t\t\tposition: 'MTF_seealso',\n\t\t\t\t\t\tadScheduleId: '',\n\t\t\t\t\t\thbVideoActive: false,\n\t\t\t\t\t\tisCM: false,\n\t\t\t\t\t\tmidrollActive: false,\n\t\t\t\t\t\tminTimeMidroll: false,\n\t\t\t\t\t});\n\t\t\t\t\t\n\t\t\t\tvar manager = AufVideo.Manager.get({\n\t\t\tlabels: {\n\t\t\t\tnextVideo: 'Next video in #SECONDS#'\n\t\t\t}\n\t\t});\n\n\t\tvar store = AufVideo.Store.get();\n\t\tstore.addInstance('af_player_60253f243e7451_42269561', {\n\t\t\tlogger: logger,\n\t\t\tplayer: player,\n\t\t\tadPlayer: adPlayer,\n\t\t\tcontext: 'ESI',\n\t\t\tposition: 'MTF_seealso',\n\t\t\tautoplay: {\n\t\t\t\tstartOnSight: true,\n\t\t\t},\n\t\t\tsticky: {\n\t\t\t\tstickTo: 'bottom-right',\n\t\t\t\tblockedCookieTtl: '',\n\t\t\t\twidth: false,\n\t\t\t\tshowBackToOriginalPositionButton: true\t\t\t}\n\t\t});\n\t})();\n\u003c/script>\n\n","serverSideRendered":true};document.currentScript.remove();</script><div><!-- ESI - VideoPlayer : http://www.netmums.com/reloaded/esi/videoplayer.php?device=desktop&position=MTF_seealso&startOnSight=1&stickTo=bottom-right&videoid=538148 -->

<script crossorigin="anonymous" src="https://static.afcdn.com/relnm/concat.js?PCM-lVLBbsIwDP0ajqsok7jtgCZtmsQBtolr5bamBNI4ipOh_v0cqkKLEBQpSl6d9_oc2xs0JbnJ7EOZHTrlwRQoX6xqq_HrEkv2_PZOtSWDxgvhT5VIKw0NRrE9ARaUQQnWoxvL3x9bHPk77y1PXhcSllWUJjnfFlRLSKvcgVMYhc36W6vl2t83gnL1ZGp9RT-5TVcoSYULctiDGXuHUDMaxqRW5r6Dd1AcWoPuB5tI-I1xZarR6s_FEzqrQ6VMlJWYh6o7x4rYq-LQnMFYGQRPsYY9OFpqQDdiNsCDTkDYyl7BraCmHHQm9a3JZGI84GgqgXdtCSg8nO7glebRXZEETm1ZUlVdjY4NOYdcQJrMk-k58MLSPY2ezICd05HxArrBunopbG8YdVc_yKzowTyylwm8T6mlAa1HMpvOUllpKsc_" type="text/javascript"></script>
<script type="text/javascript">
 window.afSession = window.afSession || new AufSession();
 </script>
<script type="text/javascript">
 window.afLogger = window.afLogger || new Aflog.Logger({
 context: {
 revision: '097f376dfff2902df245542638f65d9f32d54955'
 }
 });
 af_addEvent(window, 'beforeunload', function() {
 afLogger.sendLogsToBigQuery();
 }, false);
 </script>
<script type="text/javascript">
 if (Math.random() < 0.1) {
 (function() {
 function hsLogError(message, category, sampling) {
 var d = message.split('').reverse().join('');
 new Image().src = 'https://aufeminin.netmums.com/reloaded/errpix.php?'
 + 'd=' + encodeURIComponent(d)
 + (sampling ? '&s=' + sampling : '')
 + (category ? '&c=' + encodeURIComponent(category) : '');
 if (window.console && console.error) console.error('JS Catchall: ' + message);
 }

 var af_JsElementBlocked = false;
 var af_nbErrorsToReport = 10;
 window.onerror = function(message, url, line, column, error) {
 if (af_nbErrorsToReport-- < 0) return;

 var category = 'JS - window.onerror';

 if (typeof message === 'string') {
 if (message.indexOf('Script error') !== -1) {
 category += ' - external';
 }

 if (url && url.indexOf(document.domain) === -1) {
 message += ' @ ' + url;
 if (typeof line !== undefined) {
 message += ':' + line;
 }
 if (typeof column !== undefined) {
 message += ':' + column;
 }
 }
 } else if (typeof message == 'object' && message.srcElement && message.target) {
 if (message.srcElement == '[object HTMLScriptElement]' && message.target == '[object HTMLScriptElement]') {
 message = 'Error loading script';
 } else {
 message = 'Event Error - target:' + message.target + ' srcElement:' + message.srcElement;
 af_JsElementBlocked = true;
 }
 } else if (typeof message == 'object') {
 message = 'MESSAGE_OBJECT: ' + message.constructor.name;
 } else {
 message = 'MESSAGE_NOSTRING: ' + typeof message;
 }

 hsLogError(message, category);

 return true;
 };

 var af_taggingToReport = true;
 function checkTagging() {
 if (!af_taggingToReport) return;
 af_taggingToReport = false;
 if (window.top != window.self) return; // Iframe
 if (af_JsElementBlocked) return; // Errors when loading page

 if (!window.af_dataLayer && window.location.href.indexOf('doctissimo.fr') === -1) {
 hsLogError('GTM data layer missing', 'GTM', 1);
 }
 if (!window.af_ua_trackEvent) {
 hsLogError('UA track event function missing', 'GTM', 1);
 }
 }

 if (window.addEventListener) {
 window.addEventListener('load', checkTagging, false);
 } else if (window.attachEvent) {
 window.attachEvent('onload', checkTagging);
 }
 })();
 }</script>
<link href="https://cdn.jwplayer.com" rel="preconnect"/>
<link href="https://aufeminin.netmums.com" rel="preconnect"/>
<link href="https://static.afcdn.com" rel="preconnect"/>
<link href="https://assets.afcdn.com" rel="preconnect"/>
<link href="https://static.afcdn.com/relnm/concat.css?PCM-c87PLcjPS80r0S_LTEnND8hJrEwtQmbrFScXF-sZGRgZApGhIZACAA" media="all" rel="stylesheet" type="text/css"/>

<div class="af-videoplayer" id="af_player_60253f243e7451_42269561-container">
	<video class="" controls="" id="af_player_60253f243e7451_42269561" muted="" playsinline="" poster="https://assets.afcdn.com/video35/20180116/v538148_a-message-to-my-birth-mother_w720h720c1.jpg" preload="none">
					<source src="https://assets.afcdn.com/video35/20180116/v538148_a-message-to-my-birth-mother.m3u8" type="application/x-mpegURL"/>
						<source src="https://assets.afcdn.com/video35/20180116/v538148_a-message-to-my-birth-mother_HD.mp4" type="video/mp4"/>
						<source src="https://assets.afcdn.com/video35/20180116/v538148_a-message-to-my-birth-mother_sd.mp4" type="video/mp4"/>
					<p class="vjs-no-js">
			Your browser cannot play this video.		</p>
	</video>
</div>
<script>
	(function() {
				// JS logger
		var logger = new VideoLogger({
			context: {
				adPlayer: {
					name: 'JWPlayer'
				},
				player: {
					name: 'JWPlayer',
					version: 'JWP ' + jwplayer.version.substr(0,jwplayer.version.indexOf('+')),
					videoId: 538148,
					context: 'ESI',
					position: 'MTF_seealso',
				}
			}
		});
		var gaTracker = AufVideo.Tracker.GA.get();
				AufVideo.Tracker.Comscore.init();
		
						var player = new AufVideo.JWPlayer({
					id: 'af_player_60253f243e7451_42269561',
					aspectRatio: "16:9",
					muted: true,
					videoId: 538148,
					title: "A message to my birth mother",
					height: 720,
					playlist: {
						sources: [{"file":"https:\/\/assets.afcdn.com\/video35\/20180116\/v538148_a-message-to-my-birth-mother.m3u8","type":"application\/x-mpegURL"},{"file":"https:\/\/assets.afcdn.com\/video35\/20180116\/v538148_a-message-to-my-birth-mother_HD.mp4","type":"video\/mp4","label":"HD"},{"file":"https:\/\/assets.afcdn.com\/video35\/20180116\/v538148_a-message-to-my-birth-mother_sd.mp4","type":"video\/mp4","label":"SD"}],
						image: "https:\/\/assets.afcdn.com\/video35\/20180116\/v538148_a-message-to-my-birth-mother_w720h720c1.jpg"					},
				});
						player.publish('video.player.setup', { file: 'https://assets.afcdn.com/video35/20180116/v538148_a-message-to-my-birth-mother.m3u8' });
		
		var adPlayer;
					window.unfyVideoPlacement =  [];
			window.unfyVideoPlacement.push(
				{
					videoId: 538148,
					context: 'ESI',
					adUnit: '/229683329/netmums/video/preroll_MTF_seealso',
					placementId: 'preroll_mtf_seealso',
					playerName: 'jwplayer',
					adPlayerName: 'jwplayer-ads',
					playerId: 'af_player_60253f243e7451_42269561'
			});

			// Ad player configuration
			var adTargeting = ';'
				+ ';player=jwplayer'
				+ ';adplayer=jwplayer-ads'
				+ ';context=ESI'
				+ ';video=538148v';

								var adPlayer = new AufVideo.JWPlayerAds({
						id: 'af_player_60253f243e7451_42269561',
						player: player,
						adUnit: '/229683329/netmums/video/preroll_MTF_seealso',
						adTargeting: adTargeting,
						allowMultipleTinyAds: true,
						siteId: 35,
						position: 'MTF_seealso',
						adScheduleId: '',
						hbVideoActive: false,
						isCM: false,
						midrollActive: false,
						minTimeMidroll: false,
					});
					
				var manager = AufVideo.Manager.get({
			labels: {
				nextVideo: 'Next video in #SECONDS#'
			}
		});

		var store = AufVideo.Store.get();
		store.addInstance('af_player_60253f243e7451_42269561', {
			logger: logger,
			player: player,
			adPlayer: adPlayer,
			context: 'ESI',
			position: 'MTF_seealso',
			autoplay: {
				startOnSight: true,
			},
			sticky: {
				stickTo: 'bottom-right',
				blockedCookieTtl: '',
				width: false,
				showBackToOriginalPositionButton: true			}
		});
	})();
</script>

</div></div></div></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8905415"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo MuiAvatar-colorDefault"><img class="Avatarstyle__CustomImg-sc-1x0bfr6-1 cdFmmG" src="/coffeehouse/images/_next/avatar-default.png"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">Anonymous</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 10:46 AM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div>Morning <img alt=":hiya:" class="wysiwyg_smiley" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/greet024.gif" title=":hiya:"/><br/><br/>been busy again and so far have done all the ironing. Yet more washing in the machine <img alt=":roll:" class="wysiwyg_smiley" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/icon_rolleyes.gif" title=":roll:"/> Harry and I cycled into town and back about 2 miles each way so I have done my exercise for today!<br/><br/>Just going to start making my steak and kidney pies in a bit.<br/><br/>Also got to help Harry with a history project on the Normans and he has the Horrible Histories collection so should find some interesting facts in those.<br/>Will sort Harry's room this afternoon when Steve is awake as I don't want to disturb him.<br/><br/>Just having a nice low cal hot chocolate.<br/><br/>Cheryl - hope you are feeling a bit better today?<br/><br/>Andrea- I am sure you and Frazer can sort it out, he obviously really loves you and Caitlin. <img alt=":hug:" class="wysiwyg_smiley" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/hugs.gif" title=":hug:"/> Maybe he is a bit stressed at the moment.<br/><br/>Sharon- hope the mole is ok and like the sound of your bargains.<br/><br/>Off to start helping Harry and start cooking.<br/>Have a good day everyone xx</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="SearchCtastyle__Container-sc-123tvxn-0 eVhCqB"><div class="SearchCtastyle__Content-sc-123tvxn-1 iModOL"><a aria-disabled="false" class="MuiButtonBase-root MuiButton-root MuiButton-contained SearchCtastyle__SearchThreadButton-sc-123tvxn-2 iKSYmX MuiButton-disableElevation MuiButton-fullWidth" href="https://www.netmums.com/coffeehouse/search.php" tabindex="0"><span class="MuiButton-label">Search a thread</span></a></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8905520"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo"><img alt="karine j(2)" class="MuiAvatar-img" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/user/avatar/default_tns1_avatar.png?v=1613050416"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">karine j(2)</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 11:19 AM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div>Claire what is this universal credit?<br/><br/>Andrea I hope Frazer is back to himlself soon XXX<br/><br/>Samantha, hope your OH feels better really soon.<br/><br/>Dawn, would you have some recipes for healthy fish cakes? <img alt=";)" class="wysiwyg_smiley" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/icon_wink.gif" title=";)"/> I know you have most recipes.</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="SimilarThreadsstyle__SimilarThreadsContainer-m9qzem-0 kBLUtW"><h1 class="SimilarThreadsstyle__TitleContainer-m9qzem-1 cTZebo"><div class="CustomTitlestyle__TitleContainer-sc-112n8kf-0 btLhUS"><div class="CustomTitlestyle__TitleDecoration-sc-112n8kf-1 bqCWeV"><img alt="" class="CustomTitlestyle__TitleDecorationImg-sc-112n8kf-2 igONYw" src="/coffeehouse/images/_next/petales_left_blue.png"/></div><div class="CustomTitlestyle__Title-sc-112n8kf-3 daRUhb">Similar threads</div><div class="CustomTitlestyle__TitleDecoration-sc-112n8kf-1 bqCWeV"><img alt="" class="CustomTitlestyle__TitleDecorationImg-sc-112n8kf-2 igONYw" src="/coffeehouse/images/_next/petales_right_blue.png"/></div></div></h1><div class="SimilarThreadsstyle__SimilarThreadsSlider-m9qzem-3 XnEW"><a class="SimilarThreadsstyle__UnstyledLink-m9qzem-2 hgblsD" href="https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/619382-savvy-shoppers-training.html" rel="noreferrer" title="Savvy Shoppers in Training"><div class="SimilarThreadsstyle__SimilarThreadCard-m9qzem-4 bKzkOx"><p class="SimilarThreadsstyle__SimilarThreadCategory-m9qzem-6 cqHQJQ">Budgeting</p><p class="SimilarThreadsstyle__SimilarThreadCardTitle-m9qzem-5 bZfCKE">Savvy Shoppers in Training</p></div></a><a class="SimilarThreadsstyle__UnstyledLink-m9qzem-2 hgblsD" href="https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/1616448-savvy-benefit-card.html" rel="noreferrer" title="Savvy benefit card"><div class="SimilarThreadsstyle__SimilarThreadCard-m9qzem-4 bKzkOx"><p class="SimilarThreadsstyle__SimilarThreadCategory-m9qzem-6 cqHQJQ">Budgeting</p><p class="SimilarThreadsstyle__SimilarThreadCardTitle-m9qzem-5 bZfCKE">Savvy benefit card</p></div></a><a class="SimilarThreadsstyle__UnstyledLink-m9qzem-2 hgblsD" href="https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/351389-lovely-gift-bargains-pounstretcher-hobbycraft.html" rel="noreferrer" title="Lovely gift bargains at Pounstretcher &amp; Hobbycraft"><div class="SimilarThreadsstyle__SimilarThreadCard-m9qzem-4 bKzkOx"><p class="SimilarThreadsstyle__SimilarThreadCategory-m9qzem-6 cqHQJQ">Budgeting</p><p class="SimilarThreadsstyle__SimilarThreadCardTitle-m9qzem-5 bZfCKE">Lovely gift bargains at Pounstretcher &amp; Hobbycraft</p></div></a><a class="SimilarThreadsstyle__UnstyledLink-m9qzem-2 hgblsD" href="https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/593903-how-much-do-you-pay-per-month-water.html" rel="noreferrer" title="How much do you all pay per month for water?"><div class="SimilarThreadsstyle__SimilarThreadCard-m9qzem-4 bKzkOx"><p class="SimilarThreadsstyle__SimilarThreadCategory-m9qzem-6 cqHQJQ">Budgeting</p><p class="SimilarThreadsstyle__SimilarThreadCardTitle-m9qzem-5 bZfCKE">How much do you all pay per month for water?</p></div></a></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8905523"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo MuiAvatar-colorDefault"><img class="Avatarstyle__CustomImg-sc-1x0bfr6-1 cdFmmG" src="/coffeehouse/images/_next/avatar-default.png"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">Anonymous</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 11:19 AM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div>Hi all<br/><br/>Hugs to those who need them ! <img alt=":hug:" class="wysiwyg_smiley" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/hugs.gif" title=":hug:"/><br/><br/><br/>Just staying in today, still feel a bit rotten. Have swept the kitchen floor, put some washing away and dusted all of upstairs.<br/><br/>Got an online sainsburys shop coming after lunch so DS can help me put the shopping away later !<br/><br/>Not much else to report ! xxxx</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8905953"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo MuiAvatar-colorDefault"><img class="Avatarstyle__CustomImg-sc-1x0bfr6-1 cdFmmG" src="/coffeehouse/images/_next/avatar-default.png"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">Anonymous</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 12:49 PM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div>Hi everyone.<br/>Andrea, sometimes men need a break, they don't function like us mummies. A few days at his mums might do him good and help recharge his batteries. Xx<br/><br/>Elise, what picture are you making? <br/><br/>Sam I sympathise Hun I only have 40% of my right lung that works. I have no choice than to sleep sitting up when I'm like this. I've had brittle asthma for years. You get used to it but it never seems to get easier lol. <br/><br/>Sharon I hope you don't the hypochondriac look I seem to get lately. Sorry to hear your poorly Hun. Get better soon xxx<br/><br/>Dawn, how are you today lovely? <br/><br/>Claire not sure if it helps but for meatballs I buy mince add Oxo and onions and make into balls then add either chopped tomatoes or passata a little garlic, tomato purée and Italian herbs. Fry off the meatballs first with extra chopped onion and peppers then add the rest. Bring to the boil then simmer until cooked. I serve either alone or cook pasta and place pasta in a dish then top with the meatballs and sauce and top that with cheese then bake at about 180 until cheese bubbling. <br/><br/>Michelle, put your feet up after shopping arrives. Xx<br/><br/>Debbie, what are you cooking? <br/><br/>Karine, I have black slate in my kitchen and it drives me mad. How's the weather there? <br/><br/>Susan, what sort of auction was it? I love a good sale.<br/><br/>Jade, can you come and do my garden? It's such a mess. Need to get out there once the warmer weather comes and I'm feeling better. <br/><br/><br/>Sorry if I've missed anyone. Where is Kayleigh? Not seen her on here for a bit, or angel. Wonder if she's had the baby yet??<br/><br/>I'm off to make some lunch for ds then try to rest a bit. Steroids are really helping today but I still get short of breath with any exertion.</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="InterPostComponentstyle__AdBannerContainer-e122kk-0 fmAZHQ"><div data-ad="dyninfeed"></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8905984"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo"><img alt="Jennie W(29)" class="MuiAvatar-img" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/user/avatar/default_tns1_avatar.png?v=1613050416"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">Jennie W(29)</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 12:54 PM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div>Afternoon:hiya:<br/><br/>Well it's Phoebe's 4th bday! We bought her a Barbie Head with hair accessories &amp; a Hello Kitty Drawing Case (so she doesn't pinch Lewis';))Got her a floor jigsaw from Lewis, SS got her a load of colouring things that she will enjoy doing then my dad got her a Little Pony.  I put a status on FB &amp; everyone's commented! We are having a little tea party for her so I have spent all morning making chilli, making tuna &amp; egg mayo (will make the sandwiches in an hour or so!), cleaning up from the mess I made:lol: then made some choc rice crispies buns (&amp; cleaned up from that mess:roll:) Just need to clean up from dinner. Also done a towel wash &amp; will hang that out after this then need to clean the bathroom while I'm up there! <br/><br/>Andrea-Sorry to hear about you &amp; Frazer. Your'e more than welcome to PM me xxx<br/><br/>Yesterday spent £77 in Morrisons on bits for the party, still need to get some meat but will get sent Stu up for some on Wednesday! He's out &amp; about so hoping he'll be home soon! Kids are playing on the Wii while I'm trying to tidy the house:roll: Have cleaned the porch so will spray it with air freshener before everyone turns up!<br/><br/>Right better get on with tidying the bathroom x</div></div><div class="PostCardSignaturestyle__Container-zbxqg-0 hiqgTP"><button class="MuiButtonBase-root MuiButton-root MuiButton-text PostCardSignaturestyle__SignatureButton-zbxqg-1 deFYH MuiButton-textSecondary MuiButton-textSizeSmall MuiButton-sizeSmall MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Signature<span class="MuiButton-endIcon MuiButton-iconSizeSmall"><svg class="PostCardSignaturestyle__CustomExpandIcon-zbxqg-2 beynBk" height="12" viewBox="0 0 21 12" width="21" xmlns="http://www.w3.org/2000/svg"><path d="M10 11.79a1.76 1.76 0 0 1-1.26-.52L.52 3.05a1.8 1.8 0 0 1 0-2.53 1.8 1.8 0 0 1 2.53 0l7 7 7-6.95a1.79 1.79 0 1 1 2.53 2.53l-8.22 8.22a1.76 1.76 0 0 1-1.36.47z"></path></svg></span></span></button><div class="MuiCollapse-container MuiCollapse-hidden" style="min-height:0px"><div class="MuiCollapse-wrapper"><div class="MuiCollapse-wrapperInner">DS- 07/06/05

DD-18/02/09</div></div></div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8906012"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo MuiAvatar-colorDefault"><img class="Avatarstyle__CustomImg-sc-1x0bfr6-1 cdFmmG" src="/coffeehouse/images/_next/avatar-default.png"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">Anonymous</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 1:00 PM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div>cheryl.... the auction is an outdoor one, just a load of crap most weeks lol, but i did get OH an old 2gallon petrol can one week for £7.... as he collects them got way too many now tho that we need to sell them as we want to move to a bungalow as OH has a back condition thats getting worse....</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8906243"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo MuiAvatar-colorDefault"><img class="Avatarstyle__CustomImg-sc-1x0bfr6-1 cdFmmG" src="/coffeehouse/images/_next/avatar-default.png"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">Anonymous</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 1:46 PM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div class="DesktopQuotedPostCardstyle__Container-sc-1opvjiu-0 ddwMKL"><div class="DesktopQuotedPostCardstyle__ContentContainer-sc-1opvjiu-1 fmZwVl"><div class="DesktopQuotedPostCardstyle__Header-sc-1opvjiu-2 iSqKAc"><svg class="DesktopQuotedPostCardstyle__CustomReplyIcon-sc-1opvjiu-3 jroQyO" height="24" viewBox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg"><path d="M16 10H7.081l2.7-3.375a1 1 0 0 0-1.562-1.25l-4 5a1.001 1.001 0 0 0 0 1.25l4 5a.998.998 0 0 0 1.406.156 1 1 0 0 0 .156-1.406L7.081 12H16c.551 0 1 .449 1 1v5a1 1 0 0 0 2 0v-5c0-1.654-1.346-3-3-3"></path></svg><span class="DesktopQuotedPostCardstyle__Title-sc-1opvjiu-4 chMnry">In answer to</span><div class="DesktopQuotedPostCardstyle__TooltipZone-sc-1opvjiu-7 jBKeFF"><span class="DesktopQuotedPostCardstyle__Author-sc-1opvjiu-5 iVeKOx">Anonymous</span></div></div><div>cheryl.... the auction is an outdoor one, just a load of crap most weeks lol, but i did get OH an old 2gallon petrol can one week for £7.... as he collects them got way too many now tho that we need to sell them as we want to move to a bungalow as OH has a back condition thats getting worse....</div></div></div><div><br/><br/><br/>Up until this year the war and peace show has been hosted not far from me. They sell hundreds of these there! Your oh would be in his element lol.<br/><br/>Jennie, happy birthday to phoebe. I hope you all have a lovely day. X</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="InterPostComponentstyle__AdBannerContainer-e122kk-0 fmAZHQ"><div data-ad="dyninfeed"></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8906866"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo MuiAvatar-colorDefault"><img class="Avatarstyle__CustomImg-sc-1x0bfr6-1 cdFmmG" src="/coffeehouse/images/_next/avatar-default.png"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">Anonymous</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 4:06 PM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div>Jennie - Happy birthday to Phoebe! :partytime::birthday: Hope you have a lovely tea party x<br/><br/>Cheryl - glad that you are feeling a bit better today. Making HM Steak and kidney pies. Meat etc in slow cooker and I am going to make the pies with pastry topping in a minute and serve with mashed potatoes and veg. Boys having something different as they don't like it.<br/><br/>Had a bit of a rest this afternoon. Like being at home <img alt=":D" class="wysiwyg_smiley" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/icon_biggrin.gif" title=":D"/></div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="SimilarThreadsstyle__SimilarThreadsContainer-m9qzem-0 kBLUtW"><h1 class="SimilarThreadsstyle__TitleContainer-m9qzem-1 cTZebo"><div class="CustomTitlestyle__TitleContainer-sc-112n8kf-0 btLhUS"><div class="CustomTitlestyle__TitleDecoration-sc-112n8kf-1 bqCWeV"><img alt="" class="CustomTitlestyle__TitleDecorationImg-sc-112n8kf-2 igONYw" src="/coffeehouse/images/_next/petales_left_blue.png"/></div><div class="CustomTitlestyle__Title-sc-112n8kf-3 daRUhb">Similar threads</div><div class="CustomTitlestyle__TitleDecoration-sc-112n8kf-1 bqCWeV"><img alt="" class="CustomTitlestyle__TitleDecorationImg-sc-112n8kf-2 igONYw" src="/coffeehouse/images/_next/petales_right_blue.png"/></div></div></h1><div class="SimilarThreadsstyle__SimilarThreadsSlider-m9qzem-3 XnEW"><a class="SimilarThreadsstyle__UnstyledLink-m9qzem-2 hgblsD" href="https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/1028965-what-do-you-your-unwanted-baby-items.html" rel="noreferrer" title="What do you with all your unwanted baby items?"><div class="SimilarThreadsstyle__SimilarThreadCard-m9qzem-4 bKzkOx"><p class="SimilarThreadsstyle__SimilarThreadCategory-m9qzem-6 cqHQJQ">Budgeting</p><p class="SimilarThreadsstyle__SimilarThreadCardTitle-m9qzem-5 bZfCKE">What do you with all your unwanted baby items?</p></div></a><a class="SimilarThreadsstyle__UnstyledLink-m9qzem-2 hgblsD" href="https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/943443-started-20p.html" rel="noreferrer" title="It all started with 20p :-)"><div class="SimilarThreadsstyle__SimilarThreadCard-m9qzem-4 bKzkOx"><p class="SimilarThreadsstyle__SimilarThreadCategory-m9qzem-6 cqHQJQ">Budgeting</p><p class="SimilarThreadsstyle__SimilarThreadCardTitle-m9qzem-5 bZfCKE">It all started with 20p :-)</p></div></a><a class="SimilarThreadsstyle__UnstyledLink-m9qzem-2 hgblsD" href="https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/853705-where-good-charity-shops-hiding.html" rel="noreferrer" title="where are all the good charity shops hiding??"><div class="SimilarThreadsstyle__SimilarThreadCard-m9qzem-4 bKzkOx"><p class="SimilarThreadsstyle__SimilarThreadCategory-m9qzem-6 cqHQJQ">Budgeting</p><p class="SimilarThreadsstyle__SimilarThreadCardTitle-m9qzem-5 bZfCKE">where are all the good charity shops hiding??</p></div></a></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8907311"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo MuiAvatar-colorDefault"><img class="Avatarstyle__CustomImg-sc-1x0bfr6-1 cdFmmG" src="/coffeehouse/images/_next/avatar-default.png"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">Anonymous</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 6:02 PM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div>Cheryl your meatball recipe is great, and putting it all in one pot on the oven sounds tasty too!<br/><br/>Karine, universal credit is the new benefit system. It means all benefits will be under one name. EG if you get housing benefit and jsa, it will all come in one payment and housing ben wont be paid by the local council it will all be paidf by government I think. <br/><br/>Samantha I didnt realised it compiled all payments to monthly! We only get tax credits but recenbtly got them changed to weekly which has been much better, been able to keep on top of our food shopping and means if dh gets called to work (he is a supply lecturer) we wouldnt have to worry about having petrol money. It will be a real struggle to get food shopping if we go back to onthly <img alt=":(" class="wysiwyg_smiley" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/icon_sad.gif" title=":("/><br/><br/><br/>Jennie - happy birthday Pheobe!<br/><br/>BIL gone to pub with a friend, just me and the kids. theyre having dinner, mines is cooking then going for a shower and put my jammies on. Work was manic today, i dont know what everyone else was doing but I saw 12 patients and done 10 sales. Felt like I never stopped all day!</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="InterPostComponentstyle__MarginableContainer-e122kk-1 bLGqZf"><div class="NewThreadCtastyle__Container-il2ii3-0 awMNh"><div class="NewThreadCtastyle__Content-il2ii3-3 lhTTCP"><div class="NewThreadCtastyle__CustomTitleContainer-il2ii3-1 efiGch">Can't find your answer ?</div><button class="MuiButtonBase-root MuiButton-root MuiButton-contained NewThreadCtastyle__CreateThreadButton-il2ii3-2 laLnJE MuiButton-disableElevation MuiButton-fullWidth" tabindex="0" type="button"><span class="MuiButton-label">Create a thread</span></button></div></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8908156"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo MuiAvatar-colorDefault"><img class="Avatarstyle__CustomImg-sc-1x0bfr6-1 cdFmmG" src="/coffeehouse/images/_next/avatar-default.png"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">Anonymous</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 8:52 PM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div class="DesktopQuotedPostCardstyle__Container-sc-1opvjiu-0 ddwMKL"><div class="DesktopQuotedPostCardstyle__ContentContainer-sc-1opvjiu-1 fmZwVl"><div class="DesktopQuotedPostCardstyle__Header-sc-1opvjiu-2 iSqKAc"><svg class="DesktopQuotedPostCardstyle__CustomReplyIcon-sc-1opvjiu-3 jroQyO" height="24" viewBox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg"><path d="M16 10H7.081l2.7-3.375a1 1 0 0 0-1.562-1.25l-4 5a1.001 1.001 0 0 0 0 1.25l4 5a.998.998 0 0 0 1.406.156 1 1 0 0 0 .156-1.406L7.081 12H16c.551 0 1 .449 1 1v5a1 1 0 0 0 2 0v-5c0-1.654-1.346-3-3-3"></path></svg><span class="DesktopQuotedPostCardstyle__Title-sc-1opvjiu-4 chMnry">In answer to</span><div class="DesktopQuotedPostCardstyle__TooltipZone-sc-1opvjiu-7 jBKeFF"><span class="DesktopQuotedPostCardstyle__Author-sc-1opvjiu-5 iVeKOx">Anonymous</span></div></div><div><br/><br/><br/>Up until this year the war and peace show has been hosted not far from me. They sell hundreds of these there! Your oh would be in his element lol.<br/><br/>Jennie, happy birthday to phoebe. I hope you all have a lovely day. X</div></div></div><div><br/><br/><br/>where abouts is that?.. god he would be like a kid in a sweet shop lol</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="InterPostComponentstyle__AdBannerContainer-e122kk-0 fmAZHQ"><div data-ad="dyninfeed"></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8908164"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo MuiAvatar-colorDefault"><img class="Avatarstyle__CustomImg-sc-1x0bfr6-1 cdFmmG" src="/coffeehouse/images/_next/avatar-default.png"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">Anonymous</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 8:53 PM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div><img alt=":hiya:" class="wysiwyg_smiley" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/greet024.gif" title=":hiya:"/> Not long got back from the fair it was brilliant:D The children all had a brilliant time so did Gareth.<br/><br/>Bought them all food there for they're dinner,because we bought so much the guy that owned the food van gave Gareth 2 large bags of Candy floss for the children for free:D<br/><br/>They had a go on a few of the stalls as well and won some prizes.<br/>Jasmine and Alanha are dare devils they take after they're father!They were going on all the big scary rides:yuk:<br/><br/>I had 30 plants left over after doing the back garden yesterday so before we went to the fair I did the weeding out the front and planted the plants all out.Swept all the paths and patio out the back garden as well and tidied the bedrooms and did 3 loads of laundry.<br/><br/>Samantha I hope your OH feels better soon and that little mans leg is all better soon.xx<br/><br/>Deborah glad you managed to have some rest:D xx<br/><br/>Karine this one looks nice <a class="cLink" href="http://www.bbcgoodfood.com/recipes/3627/simple-fish-cakes">Simple fish cakes recipe - Recipes - BBC Good Food</a> xx<br/><br/>Michelle I hope you managed to have a rest and hope you feel better soon.xx<br/><br/>Cheryl glad the steroids are helping.Any news if work can help with getting you reffered somewhere for your chest.xx<br/><br/>Jennie,Happy 4th Birthday Phoebe :D:hiya: xx</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8908185"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo MuiAvatar-colorDefault"><img class="Avatarstyle__CustomImg-sc-1x0bfr6-1 cdFmmG" src="/coffeehouse/images/_next/avatar-default.png"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">Anonymous</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 8:55 PM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div>made a chicken stew with left over chicken and blended some up for lucy before adding the gravy.... i used one of those new bisto stock melts and ive got to say they are so tasty.. had to blend mine up too, god i could kill for a bar of chocolate and a picee of bakewell tart lol  has anyone else tried them?<br/><br/>also has anyone got a quick and easy chicken curry recipe i can do in slow cooker?</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="SimilarThreadsstyle__SimilarThreadsContainer-m9qzem-0 kBLUtW"><h1 class="SimilarThreadsstyle__TitleContainer-m9qzem-1 cTZebo"><div class="CustomTitlestyle__TitleContainer-sc-112n8kf-0 btLhUS"><div class="CustomTitlestyle__TitleDecoration-sc-112n8kf-1 bqCWeV"><img alt="" class="CustomTitlestyle__TitleDecorationImg-sc-112n8kf-2 igONYw" src="/coffeehouse/images/_next/petales_left_blue.png"/></div><div class="CustomTitlestyle__Title-sc-112n8kf-3 daRUhb">Similar threads</div><div class="CustomTitlestyle__TitleDecoration-sc-112n8kf-1 bqCWeV"><img alt="" class="CustomTitlestyle__TitleDecorationImg-sc-112n8kf-2 igONYw" src="/coffeehouse/images/_next/petales_right_blue.png"/></div></div></h1><div class="SimilarThreadsstyle__SimilarThreadsSlider-m9qzem-3 XnEW"><a class="SimilarThreadsstyle__UnstyledLink-m9qzem-2 hgblsD" href="https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/452959-summer-sale-extra-10-netmums.html" rel="noreferrer" title="Summer sale and an extra 10% for all netmums"><div class="SimilarThreadsstyle__SimilarThreadCard-m9qzem-4 bKzkOx"><p class="SimilarThreadsstyle__SimilarThreadCategory-m9qzem-6 cqHQJQ">Budgeting</p><p class="SimilarThreadsstyle__SimilarThreadCardTitle-m9qzem-5 bZfCKE">Summer sale and an extra 10% for all netmums</p></div></a><a class="SimilarThreadsstyle__UnstyledLink-m9qzem-2 hgblsD" href="https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/703036-home-bargains-have-rug-doctorcarpet-shampoo-other-sprays-etc-mega-ch.html" rel="noreferrer" title="Home Bargains have Rug Doctorcarpet shampoo and all the other sprays etc mega ch"><div class="SimilarThreadsstyle__SimilarThreadCard-m9qzem-4 bKzkOx"><p class="SimilarThreadsstyle__SimilarThreadCategory-m9qzem-6 cqHQJQ">Budgeting</p><p class="SimilarThreadsstyle__SimilarThreadCardTitle-m9qzem-5 bZfCKE">Home Bargains have Rug Doctorcarpet shampoo and all the other sprays etc mega ch</p></div></a><a class="SimilarThreadsstyle__UnstyledLink-m9qzem-2 hgblsD" href="https://www.netmums.com/coffeehouse/money-advice-1127/budgeting-609/309280-money-codes-theme-parks-courtesy-martins-money-tips.html" rel="noreferrer" title="Money of codes for ALL theme parks - courtesy of Martins Money Tips"><div class="SimilarThreadsstyle__SimilarThreadCard-m9qzem-4 bKzkOx"><p class="SimilarThreadsstyle__SimilarThreadCategory-m9qzem-6 cqHQJQ">Budgeting</p><p class="SimilarThreadsstyle__SimilarThreadCardTitle-m9qzem-5 bZfCKE">Money of codes for ALL theme parks - courtesy of Martins Money Tips</p></div></a></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8908195"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo"><img alt="LAURA D(17)" class="MuiAvatar-img" src="https://cdn.prod.nmums.doc-media.fr/netmums_awsprod/avatars/8/3/6/159836/825/tns1-825.png?v=1"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">LAURA D(17)</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 8:57 PM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div>Evening all <br/><br/>Andrea hope ur ok:hug:<br/>Karine were all much better now thanks well apart from ds who has a nasty cough so going to phone docs and make him an appointment tomo<br/>Cheryl hope u feel better soon <img alt=":hug:" class="wysiwyg_smiley" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/hugs.gif" title=":hug:"/><br/>Hope everyone else is ok <img alt=":hug:" class="wysiwyg_smiley" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/hugs.gif" title=":hug:"/><br/><br/>Everyone that's on half term this week I hope u all have a lovely week.<br/>Weather here has been lovely hope it lasts til my day off. NSD today, hoping to have NSD tomo too well and wed too as fri will prob end up being a HSD as we're going to watch Disney on Ice and my niece is coming with us too. We're going to go back to having permanent fri night movie nights as this seemed to fizzle out for a bit.</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div><div class="InterPostComponentstyle__AdBannerContainer-e122kk-0 fmAZHQ"><div data-ad="dyninfeed"></div></div><div class="DesktopPostCardstyle__PostContainer-t45zrm-5 iiMZYv"><div class="DesktopPostCardHeaderstyle__Container-sc-17t45kd-1 jkmxUM" id="8908277"><div class="DesktopPostCardHeaderstyle__UserContainer-sc-17t45kd-2 btrLea"><div class="DesktopPostCardHeaderstyle__TooltipZone-sc-17t45kd-4 ewmuHh"><div class="DesktopPostCardHeaderstyle__AvatarContainer-sc-17t45kd-6 iVvXSp"><div class="MuiAvatar-root MuiAvatar-circle Avatarstyle__CustomAvatar-sc-1x0bfr6-0 cQGPpo"><img alt="LAURA D(17)" class="MuiAvatar-img" src="https://cdn.prod.nmums.doc-media.fr/netmums_awsprod/avatars/8/3/6/159836/825/tns1-825.png?v=1"/></div></div><div class="DesktopPostCardHeaderstyle__InfoContainer-sc-17t45kd-5 jTVLai"><div class="DesktopPostCardHeaderstyle__UserPseudo-sc-17t45kd-0 cSgIid">LAURA D(17)</div><div class="DesktopPostCardHeaderstyle__PostDate-sc-17t45kd-7 cSgZVd">2/18/2013 at 9:09 PM</div></div></div></div><div class="DesktopPostCardHeaderstyle__Actions-sc-17t45kd-8 cngpQz"><button aria-label="Open post actions" class="MuiButtonBase-root MuiIconButton-root DesktopPostCardHeaderstyle__CustomMoreButton-sc-17t45kd-12 fXVIbQ" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="DesktopPostCardHeaderstyle__CustomMoreIcon-sc-17t45kd-11 hxFQiP" height="4" viewBox="0 0 23 4" width="23" xmlns="http://www.w3.org/2000/svg"><path d="M11.5 0c1.12 0 2.03.896 2.03 2s-.91 2-2.03 2c-1.12 0-2.03-.896-2.03-2s.91-2 2.03-2zm9.47 0C22.09 0 23 .896 23 2s-.91 2-2.03 2c-1.12 0-2.029-.896-2.029-2s.91-2 2.03-2zM2.03 0c1.12 0 2.029.896 2.029 2s-.91 2-2.03 2C.91 4 0 3.104 0 2s.91-2 2.03-2z"></path></svg></span></button></div></div><div class="DesktopPostCardstyle__ThreadContent-t45zrm-3 gIugAn"><div>Sorry forgot<br/>Jennifer hope phoebe had a lovely day<br/><br/>Dawn sounds like you've all had a fab afternoon, what plants have you planted? My nan always tells me it's to early to start planting.<br/>Susan chicken stew yummy I haven't had that for awhile</div></div><div class="DesktopPostCardFooterstyle__Container-sc-1qzm02m-0 rGdlb"><div class="DesktopPostCardFooterstyle__ActionsContainer-sc-1qzm02m-1 eurmgj"><div class="LikeCounterstyle__Container-kp9n2u-0 dqJGEV"><div class="LikeCounterstyle__ButtonContainer-kp9n2u-1 cEbOEy"><button class="MuiButtonBase-root MuiIconButton-root LikeCounterstyle__CustomButton-kp9n2u-2 biZUop" tabindex="0" type="button"><span class="MuiIconButton-label"><svg class="LikeCounterstyle__CustomLikeIcon-kp9n2u-3 hxtTFu" height="19" viewBox="-1.4 0 23 20" width="20" xmlns="http://www.w3.org/2000/svg"><path d="M10 19a1 1 0 0 1-.71-.31l-7.76-8.24a5.77 5.77 0 0 1 0-7.83 5 5 0 0 1 7.38 0L10 3.77l1.08-1.15a5 5 0 0 1 7.39 0 5.77 5.77 0 0 1 0 7.83l-7.76 8.24A1 1 0 0 1 10 19"></path></svg></span></button></div>0 like</div><button class="MuiButtonBase-root MuiButton-root MuiButton-outlined DesktopPostCardstyle__ReplyButton-t45zrm-4 btHWAz MuiButton-outlinedPrimary MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">Reply</span></button></div></div></div></div><fieldset class="ActiveThreadsstyle__CustomFieldset-sc-1as8dk-0 ezjMjR"><legend class="ActiveThreadsstyle__CustomLegend-sc-1as8dk-1 bLwFTq"><div class="CustomTitlestyle__TitleContainer-sc-112n8kf-0 btLhUS"><div class="CustomTitlestyle__TitleDecoration-sc-112n8kf-1 bqCWeV"><img alt="" class="CustomTitlestyle__TitleDecorationImg-sc-112n8kf-2 igONYw" src="/coffeehouse/images/_next/petales_left_orange.png"/></div><div class="CustomTitlestyle__Title-sc-112n8kf-3 daRUhb">Most active chat</div><div class="CustomTitlestyle__TitleDecoration-sc-112n8kf-1 bqCWeV"><img alt="" class="CustomTitlestyle__TitleDecorationImg-sc-112n8kf-2 igONYw" src="/coffeehouse/images/_next/petales_right_orange.png"/></div></div></legend><div class="ActiveThreadsstyle__Content-sc-1as8dk-2 Ikwdt"><a class="MuiTypography-root MuiLink-root MuiLink-underlineHover ActiveThreadstyle__CustomLink-sc-1pce28s-1 cHsuON MuiTypography-colorPrimary" href="https://www.netmums.com/coffeehouse/diets-1061/losing-weight-619/1935417-see-my-video.html"><div class="ActiveThreadstyle__LinkContent-sc-1pce28s-0 AumzX">see my video !</div></a><a class="MuiTypography-root MuiLink-root MuiLink-underlineHover ActiveThreadstyle__CustomLink-sc-1pce28s-1 cHsuON MuiTypography-colorPrimary" href="https://www.netmums.com/coffeehouse/becoming-mum-ttc-64/am-i-pregnant-874/1935463-held-me-urine-over-4hrs-got-result-my-pregnancy-test.html"><div class="ActiveThreadstyle__LinkContent-sc-1pce28s-0 AumzX">held me urine in for over 4hrs and got this result on my pregnancy test ?</div></a><a class="MuiTypography-root MuiLink-root MuiLink-underlineHover ActiveThreadstyle__CustomLink-sc-1pce28s-1 cHsuON MuiTypography-colorPrimary" href="https://www.netmums.com/coffeehouse/life-504/hair-beauty-530/1935462-looking-older-than-i-am.html"><div class="ActiveThreadstyle__LinkContent-sc-1pce28s-0 AumzX">Looking older than I am</div></a><a class="MuiTypography-root MuiLink-root MuiLink-underlineHover ActiveThreadstyle__CustomLink-sc-1pce28s-1 cHsuON MuiTypography-colorPrimary" href="https://www.netmums.com/coffeehouse/becoming-mum-pregnancy-996/pregnancy-1213/1935459-4-weeks-1-day-hcg-levels.html"><div class="ActiveThreadstyle__LinkContent-sc-1pce28s-0 AumzX">4 weeks 1 day hcg levels!</div></a><a class="MuiTypography-root MuiLink-root MuiLink-underlineHover ActiveThreadstyle__CustomLink-sc-1pce28s-1 cHsuON MuiTypography-colorPrimary" href="https://www.netmums.com/coffeehouse/home-garden-194/cleaning-14/1935453-see-my-video.html"><div class="ActiveThreadstyle__LinkContent-sc-1pce28s-0 AumzX">see my video !</div></a></div></fieldset><div class="Paginationstyle__Container-d53ifi-3 ioFwmU"><div class="Paginationstyle__RowContainer-d53ifi-0 crjKLc"><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">1</a></div><div class="MuiPaginationItem-root MuiPaginationItem-ellipsis MuiPaginationItem-sizeSmall">…</div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-680.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">680</a></div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-681.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">681</a></div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-682.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">682</a></div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-683.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">683</a></div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-684.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">684</a></div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-685.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">685</a></div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-686.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">686</a></div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-687.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">687</a></div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 eRClxI" selected=""><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-688.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">688</a></div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-689.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">689</a></div></div><button class="MuiButtonBase-root MuiButton-root MuiButton-text Paginationstyle__SeeMoreButton-d53ifi-2 igTqKU MuiButton-textSizeSmall MuiButton-sizeSmall MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label">See more</span></button><div class="Paginationstyle__Optional-d53ifi-1 fsDQEW"><div class="Paginationstyle__RowContainer-d53ifi-0 crjKLc"><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-700.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">700</a></div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-800.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">800</a></div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-900.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">900</a></div><div class="PaginationItemstyle__Container-sc-1fdlb6e-0 idnEnj"><a class="PaginationItemstyle__TemporaryUnstyledA-sc-1fdlb6e-2 eCvIhI" href="/coffeehouse/money-advice-1127/budgeting-609/551294-you-lovely-savvy-shoppers-954.html"></a><a class="PaginationItemstyle__LinkContent-sc-1fdlb6e-1 cLorOg">954</a></div></div></div></div><div class="ForumFooterstyle__SmallCtaContainer-yu3eug-0 kIISjG"><a aria-disabled="false" class="MuiButtonBase-root MuiButton-root MuiButton-contained MuiButton-containedSizeSmall MuiButton-sizeSmall MuiButton-disableElevation" href="https://www.netmums.com/info/netmums-faqs" tabindex="0"><span class="MuiButton-label"><span class="MuiButton-startIcon MuiButton-iconSizeSmall"><svg height="15" viewBox="0 0 15 15" width="15" xmlns="http://www.w3.org/2000/svg"><path d="M7.5 10.5a.75.75 0 1 1-.75.75.76.76 0 0 1 .75-.75zm0-7.5a2.62 2.62 0 0 1 .75 5.13V9a.75.75 0 0 1-1.5 0V7.5a.76.76 0 0 1 .75-.75 1.13 1.13 0 1 0-1.12-1.13c0 .42-.34.76-.76.76a.75.75 0 0 1-.74-.76A2.62 2.62 0 0 1 7.5 3zm0 10.5a6 6 0 1 1 0-12 6 6 0 0 1 0 12M7.5 0a7.5 7.5 0 1 0 0 15 7.5 7.5 0 0 0 0-15" fill="#111" fill-rule="evenodd"></path></svg></span>Help</span></a><a aria-disabled="false" class="MuiButtonBase-root MuiButton-root MuiButton-contained MuiButton-containedSizeSmall MuiButton-sizeSmall MuiButton-disableElevation" href="https://www.netmums.com/info/chat-rules" tabindex="0"><span class="MuiButton-label"><span class="MuiButton-startIcon MuiButton-iconSizeSmall"><svg height="15" viewBox="0 0 13 15" width="13" xmlns="http://www.w3.org/2000/svg"><path d="M2.37 12.62a.79.79 0 1 1 0-1.58h8.71v1.58H2.37zm0-11h8.71V9.5H2.37a2.3 2.3 0 0 0-.79.14V2.37a.79.79 0 0 1 .79-.75zM11.83 0H2.37A2.37 2.37 0 0 0 0 2.37v9.5a2.37 2.37 0 0 0 2.37 2.37h9.46a.79.79 0 0 0 .79-.79V.79a.79.79 0 0 0-.79-.79z" fill="#111"></path></svg></span>See chat rules</span></a></div></div><div class="ThreadDesktopstyle__Banner300AtfContainer-sc-2f61v0-3 bFUkyW"><div data-ad="300_atf"></div></div></div></div></div><div data-ad-skin-above="true" data-ad-sticky-unstick-to="#pegasus_banner_atf"><div><script>window.__REACT_ESI__ = window.__REACT_ESI__ || {}; window.__REACT_ESI__['NetmumsFooter'] = {"htmlData":"\u003cfooter class=\"nmg-footer\">\u003cdiv class=\"nmg-footer__wpr\">\u003cdiv class=\"nmg-promo\">\u003ca href=\"https://www.netmums.com/info/pregnancy-emails\" onclick=\"af_ua_trackEvent('Pregnancy', 'Link from Slice')\">\u003cpicture>\u003csource media=\"(min-width: 1024px)\" srcset=\"https://images.prismic.io/netmums/a621c65e-1631-4373-9d01-a68e687ba5ae_1024+%E2%80%93+41+weeks.png?auto=compress,format&rect=0,0,1024,256&w=1024&h=256\">\u003csource media=\"(min-width: 768px)\" srcset=\"https://images.prismic.io/netmums/48f34da8-5d8a-4eb7-8b4b-6176aaeccfa1_768.png?auto=compress,format&rect=0,0,768,256&w=768&h=256\">\u003cimg loading=\"lazy\" loading=\"lazy\" src=\"https://images.prismic.io/netmums/1b9d19e5-2a40-4c41-84a0-b031ccb52370_375.png?auto=compress,format&rect=0,0,375,256&w=375&h=256\" alt=\"Pregnancy NL banner image\">\u003c/picture>\u003c/a>\u003c/div>\u003cdiv class=\"nmg-footer__main\">\u003cdiv class=\"nmg-footer__block\">\u003ch3 class=\"nmg-footer__block__title\">About Netmums\u003c/h3>\u003cul class=\"nmg-footer__links\">\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/info/about-us\">About us\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/info/meet-the-netmums-content-team\">Meet the content team\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/info/meet-the-netmums-moderation-team\">Meet the moderation team\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/info/netmums-trusted-experts\">Our trusted experts\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/info/netmums-drop-in-clinic-parent-supporter-service\">Drop-In Clinic and PS team\u003c/a>\u003c/li>\u003c/ul>\u003c/div>\u003cdiv class=\"nmg-footer__block\">\u003ch3 class=\"nmg-footer__block__title\">FAQs and support\u003c/h3>\u003cul class=\"nmg-footer__links\">\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/support/a-z-for-help-advice-and-support\">A-Z of parenting support\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/info/netmums-faqs\">Netmums FAQs\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/info/chat-rules\">Forum chat rules\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/reviews/write-a-review\">Write a product review\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/info/contact-us\">Contact us\u003c/a>\u003c/li>\u003c/ul>\u003c/div>\u003cdiv class=\"nmg-footer__block\">\u003ch3 class=\"nmg-footer__block__title\">Media resources\u003c/h3>\u003cul class=\"nmg-footer__links\">\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/info/netmums-is-proud-to-partner-with\">Netmums: proud to partner with\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/info/netmums-recommended\">Netmums recommended\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/reviews/buying-guides/about-star-reviewers\">About star reviewers\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://advertisements.netmums.com\" target=\"_blank\">Advertise with Netmums\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__link\">\u003ca href=\"https://www.netmums.com/info/netmums-in-the-news\">Netmums in the news\u003c/a>\u003c/li>\u003c/ul>\u003c/div>\u003cdiv class=\"nmg-footer__block\">\u003ch3 class=\"nmg-footer__block__title\">Follow Netmums\u003c/h3>\u003cul class=\"nmg-footer__links nmg-footer__links--social\">\u003cli class=\"nmg-footer__social__icon nmg-footer__social__icon--facebook\">\u003ca href=\"https://www.facebook.com/Netmums\" target=\"_blank\">\u003cspan>Follow Netmums on Facebook\u003c/span>\u003csvg aria-hidden=\"true\" height=\"26\" viewBox=\"0 0 800 800\" width=\"26\" xmlns=\"http://www.w3.org/2000/svg\">\u003cpath d=\"M445 643h-90V419h-75v-87h75v-64q0-55 32-86 30-29 80-29 28 0 67 3v78h-47q-42 0-42 38v60h86l-11 87h-75v224z\"/>\u003c/svg>\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__social__icon nmg-footer__social__icon--twitter\">\u003ca href=\"https://twitter.com/Netmums\" target=\"_blank\">\u003cspan>Follow Netmums on Twitter\u003c/span>\u003csvg aria-hidden=\"true\" height=\"26\" viewBox=\"0 0 800 800\" width=\"26\" xmlns=\"http://www.w3.org/2000/svg\">\u003cpath d=\"M679 239s-21 34-55 57c7 156-107 329-314 329-103 0-169-50-169-50s81 17 163-45c-83-5-103-77-103-77s23 6 50-2c-93-23-89-110-89-110s23 14 50 14c-84-65-34-148-34-148s76 107 228 116c-22-121 117-177 188-101 37-6 71-27 71-27s-12 41-49 61c30-2 63-17 63-17z\"/>\u003c/svg>\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__social__icon nmg-footer__social__icon--pinterest\">\u003ca href=\"https://www.pinterest.co.uk/netmums/\" target=\"_blank\">\u003cspan>Follow Netmums on Pinterest\u003c/span>\u003csvg aria-hidden=\"true\" height=\"26\" viewBox=\"0 0 800 800\" width=\"26\" xmlns=\"http://www.w3.org/2000/svg\">\u003cpath d=\"M287 681c-9-3-15-77-6-115l38-163s-9-20-9-49c0-46 26-80 59-80 28 0 42 21 42 46 0 28-19 71-28 110-8 33 17 59 49 59 58 0 104-61 104-150 0-79-57-134-138-134-94 0-148 69-148 142 0 28 9.7 57.4 23 74 4 5 5 6 2 17l-8 31s-2 9.5-14 3c-41-22.3-63-78-63-126 0-104 75-199 217-199 114 0 203 81 203 190 0 113-72 205-171 205-55 0-75-38-75-38l-21 78c-11 41-47 102-56 99z\"/>\u003c/svg>\u003c/a>\u003c/li>\u003cli class=\"nmg-footer__social__icon nmg-footer__social__icon--instagram\">\u003ca href=\"https://www.instagram.com/netmums/\" target=\"_blank\">\u003cspan>Follow Netmums on Instagram\u003c/span>\u003csvg aria-hidden=\"true\" height=\"26\" viewBox=\"0 0 800 800\" width=\"26\" xmlns=\"http://www.w3.org/2000/svg\">\u003cpath d=\"M150 400c0-119 0-166 42-208s88-42 208-42 166 0 208 42 42 89 42 208 0 166-42 208-88 42-208 42-166 0-208-42-42-89-42-208zm455 0c0-114 0-148-29-176-29-29-62-29-176-29s-148 0-176 29c-29 29-29 62-29 176s0 148 29 176c29 29 62 29 176 29s148 0 176-29c29-29 29-62 29-176zM400 272a128 128 0 1 1 0 256 128 128 0 0 1 0-256zm0 211c46 0 83-37 83-83s-37-83-83-83-83 37-83 83 37 83 83 83zm163-216c0 16-13 30-30 30-16 0-30-14-30-30 0-17 14-30 30-30 17 0 30 13 30 30z\"/>\u003c/svg>\u003c/a>\u003c/li>\u003c/ul>\u003ca href=\"https://www.netmums.com/login\" class=\"nmg-footer__join\" data-role=\"none\">Join Netmums\u003c/a>\u003c/div>\u003c/div>\u003cdiv class=\"nmg-footer__secondary\">\u003cform action=\"https://www.netmums.com/newsletter-signup\" id=\"NmgFooterSignup\" class=\"nmg-footer__signup__form\" method=\"post\" accept-charset=\"utf-8\">\u003cdiv style=\"display:none;\">\u003cinput type=\"hidden\" name=\"_method\" value=\"POST\" />\u003c/div>\u003cdiv class=\"nmg-footer__signup__form__banner\">\u003cimg loading=\"lazy\" loading=\"lazy\" src=\"https://images.prismic.io/netmums/6c1bbece-b864-4fb4-8baf-9307961e6cc8_650+for+newsletter.png?auto=compress,format&rect=0,0,650,350&w=650&h=350\" alt=\"Vernon Kay on Sweat, Snot & Tears podcast\">\u003c/div>\u003cdiv class=\"footer__signup__form__wpr\">\u003cinput type=\"hidden\" name=\"emailSource\" value=\"footer\" id=\"footer_newsletter_signupSource\" />\u003cinput name=\"emailSignup\" type=\"email\" placeholder=\"Enter your email...\" class=\"nmg-footer__signup__form__field\" required=\"required\" id=\"UserEmail\" />\u003cbutton type=\"submit\" class=\"nmg-footer__signup__form__submit\">Sign up\u003c/button>\u003c/div>\u003cdiv class=\"nmg-footer__signup__form__checkbox\">\u003cinput type=\"checkbox\" name=\"emailSignup3rdParty\" value=\"1\" id=\"UserNewsletterPartnerOptin\" />\u003clabel for=\"UserNewsletterPartnerOptin\">Please tick if you would like to receive news, offers and information from our trusted and carefully selected \u003ca href=\"https://www.netmums.com/info/privacy-policy\">partners\u003c/a> that we think you might like.\u003c/label>\u003c/div>\u003c/form>\u003c/div>\u003cdiv class=\"nmg-footer__legal\">\u003ca href=\"https://www.netmums.com/info/terms-conditions\"data-role=\"none\">Terms and Conditions\u003c/a>\u003ca href=\"https://www.netmums.com/info/privacy-notice\" data-role=\"none\">Privacy Notice\u003c/a>\u003ca href=\"https://www.netmums.com/info/the-use-of-cookies-on-netmums\" data-role=\"none\">Cookie Policy\u003c/a>\u003ca href=\"javascript:Didomi.preferences.show()\" rel=\"nofollow\" data-role=\"none\">Cookie settings\u003c/a>\u003cp>&copy; 2021 Netmums Ltd\u003c/p>\u003c/div>\u003c/div>\u003c/footer>\u003cscript type=\"text/javascript\" src=\"https://www.netmums.com/min/g=mes-discussions.js&1613052748\">\u003c/script>","serverSideRendered":true};document.currentScript.remove();</script><div><footer class="nmg-footer"><div class="nmg-footer__wpr"><div class="nmg-promo"><a href="https://www.netmums.com/info/pregnancy-emails" onclick="af_ua_trackEvent('Pregnancy', 'Link from Slice')"><picture><source media="(min-width: 1024px)" srcset="https://images.prismic.io/netmums/a621c65e-1631-4373-9d01-a68e687ba5ae_1024+%E2%80%93+41+weeks.png?auto=compress,format&amp;rect=0,0,1024,256&amp;w=1024&amp;h=256"/><source media="(min-width: 768px)" srcset="https://images.prismic.io/netmums/48f34da8-5d8a-4eb7-8b4b-6176aaeccfa1_768.png?auto=compress,format&amp;rect=0,0,768,256&amp;w=768&amp;h=256"/><img alt="Pregnancy NL banner image" loading="lazy" src="https://images.prismic.io/netmums/1b9d19e5-2a40-4c41-84a0-b031ccb52370_375.png?auto=compress,format&amp;rect=0,0,375,256&amp;w=375&amp;h=256"/></picture></a></div><div class="nmg-footer__main"><div class="nmg-footer__block"><h3 class="nmg-footer__block__title">About Netmums</h3><ul class="nmg-footer__links"><li class="nmg-footer__link"><a href="https://www.netmums.com/info/about-us">About us</a></li><li class="nmg-footer__link"><a href="https://www.netmums.com/info/meet-the-netmums-content-team">Meet the content team</a></li><li class="nmg-footer__link"><a href="https://www.netmums.com/info/meet-the-netmums-moderation-team">Meet the moderation team</a></li><li class="nmg-footer__link"><a href="https://www.netmums.com/info/netmums-trusted-experts">Our trusted experts</a></li><li class="nmg-footer__link"><a href="https://www.netmums.com/info/netmums-drop-in-clinic-parent-supporter-service">Drop-In Clinic and PS team</a></li></ul></div><div class="nmg-footer__block"><h3 class="nmg-footer__block__title">FAQs and support</h3><ul class="nmg-footer__links"><li class="nmg-footer__link"><a href="https://www.netmums.com/support/a-z-for-help-advice-and-support">A-Z of parenting support</a></li><li class="nmg-footer__link"><a href="https://www.netmums.com/info/netmums-faqs">Netmums FAQs</a></li><li class="nmg-footer__link"><a href="https://www.netmums.com/info/chat-rules">Forum chat rules</a></li><li class="nmg-footer__link"><a href="https://www.netmums.com/reviews/write-a-review">Write a product review</a></li><li class="nmg-footer__link"><a href="https://www.netmums.com/info/contact-us">Contact us</a></li></ul></div><div class="nmg-footer__block"><h3 class="nmg-footer__block__title">Media resources</h3><ul class="nmg-footer__links"><li class="nmg-footer__link"><a href="https://www.netmums.com/info/netmums-is-proud-to-partner-with">Netmums: proud to partner with</a></li><li class="nmg-footer__link"><a href="https://www.netmums.com/info/netmums-recommended">Netmums recommended</a></li><li class="nmg-footer__link"><a href="https://www.netmums.com/reviews/buying-guides/about-star-reviewers">About star reviewers</a></li><li class="nmg-footer__link"><a href="https://advertisements.netmums.com" target="_blank">Advertise with Netmums</a></li><li class="nmg-footer__link"><a href="https://www.netmums.com/info/netmums-in-the-news">Netmums in the news</a></li></ul></div><div class="nmg-footer__block"><h3 class="nmg-footer__block__title">Follow Netmums</h3><ul class="nmg-footer__links nmg-footer__links--social"><li class="nmg-footer__social__icon nmg-footer__social__icon--facebook"><a href="https://www.facebook.com/Netmums" target="_blank"><span>Follow Netmums on Facebook</span><svg aria-hidden="true" height="26" viewBox="0 0 800 800" width="26" xmlns="http://www.w3.org/2000/svg"><path d="M445 643h-90V419h-75v-87h75v-64q0-55 32-86 30-29 80-29 28 0 67 3v78h-47q-42 0-42 38v60h86l-11 87h-75v224z"></path></svg></a></li><li class="nmg-footer__social__icon nmg-footer__social__icon--twitter"><a href="https://twitter.com/Netmums" target="_blank"><span>Follow Netmums on Twitter</span><svg aria-hidden="true" height="26" viewBox="0 0 800 800" width="26" xmlns="http://www.w3.org/2000/svg"><path d="M679 239s-21 34-55 57c7 156-107 329-314 329-103 0-169-50-169-50s81 17 163-45c-83-5-103-77-103-77s23 6 50-2c-93-23-89-110-89-110s23 14 50 14c-84-65-34-148-34-148s76 107 228 116c-22-121 117-177 188-101 37-6 71-27 71-27s-12 41-49 61c30-2 63-17 63-17z"></path></svg></a></li><li class="nmg-footer__social__icon nmg-footer__social__icon--pinterest"><a href="https://www.pinterest.co.uk/netmums/" target="_blank"><span>Follow Netmums on Pinterest</span><svg aria-hidden="true" height="26" viewBox="0 0 800 800" width="26" xmlns="http://www.w3.org/2000/svg"><path d="M287 681c-9-3-15-77-6-115l38-163s-9-20-9-49c0-46 26-80 59-80 28 0 42 21 42 46 0 28-19 71-28 110-8 33 17 59 49 59 58 0 104-61 104-150 0-79-57-134-138-134-94 0-148 69-148 142 0 28 9.7 57.4 23 74 4 5 5 6 2 17l-8 31s-2 9.5-14 3c-41-22.3-63-78-63-126 0-104 75-199 217-199 114 0 203 81 203 190 0 113-72 205-171 205-55 0-75-38-75-38l-21 78c-11 41-47 102-56 99z"></path></svg></a></li><li class="nmg-footer__social__icon nmg-footer__social__icon--instagram"><a href="https://www.instagram.com/netmums/" target="_blank"><span>Follow Netmums on Instagram</span><svg aria-hidden="true" height="26" viewBox="0 0 800 800" width="26" xmlns="http://www.w3.org/2000/svg"><path d="M150 400c0-119 0-166 42-208s88-42 208-42 166 0 208 42 42 89 42 208 0 166-42 208-88 42-208 42-166 0-208-42-42-89-42-208zm455 0c0-114 0-148-29-176-29-29-62-29-176-29s-148 0-176 29c-29 29-29 62-29 176s0 148 29 176c29 29 62 29 176 29s148 0 176-29c29-29 29-62 29-176zM400 272a128 128 0 1 1 0 256 128 128 0 0 1 0-256zm0 211c46 0 83-37 83-83s-37-83-83-83-83 37-83 83 37 83 83 83zm163-216c0 16-13 30-30 30-16 0-30-14-30-30 0-17 14-30 30-30 17 0 30 13 30 30z"></path></svg></a></li></ul><a class="nmg-footer__join" data-role="none" href="https://www.netmums.com/login">Join Netmums</a></div></div><div class="nmg-footer__secondary"><form accept-charset="utf-8" action="https://www.netmums.com/newsletter-signup" class="nmg-footer__signup__form" id="NmgFooterSignup" method="post"><div style="display:none;"><input name="_method" type="hidden" value="POST"/></div><div class="nmg-footer__signup__form__banner"><img alt="Vernon Kay on Sweat, Snot &amp; Tears podcast" loading="lazy" src="https://images.prismic.io/netmums/6c1bbece-b864-4fb4-8baf-9307961e6cc8_650+for+newsletter.png?auto=compress,format&amp;rect=0,0,650,350&amp;w=650&amp;h=350"/></div><div class="footer__signup__form__wpr"><input id="footer_newsletter_signupSource" name="emailSource" type="hidden" value="footer"/><input class="nmg-footer__signup__form__field" id="UserEmail" name="emailSignup" placeholder="Enter your email..." required="required" type="email"/><button class="nmg-footer__signup__form__submit" type="submit">Sign up</button></div><div class="nmg-footer__signup__form__checkbox"><input id="UserNewsletterPartnerOptin" name="emailSignup3rdParty" type="checkbox" value="1"/><label for="UserNewsletterPartnerOptin">Please tick if you would like to receive news, offers and information from our trusted and carefully selected <a href="https://www.netmums.com/info/privacy-policy">partners</a> that we think you might like.</label></div></form></div><div class="nmg-footer__legal"><a data-role="none" href="https://www.netmums.com/info/terms-conditions">Terms and Conditions</a><a data-role="none" href="https://www.netmums.com/info/privacy-notice">Privacy Notice</a><a data-role="none" href="https://www.netmums.com/info/the-use-of-cookies-on-netmums">Cookie Policy</a><a data-role="none" href="javascript:Didomi.preferences.show()" rel="nofollow">Cookie settings</a><p>© 2021 Netmums Ltd</p></div></div></footer><script src="https://www.netmums.com/min/g=mes-discussions.js&amp;1613052748" type="text/javascript"></script></div></div></div></main></div></div><script lang="js">if (typeof window.__REACT_DEVTOOLS_GLOBAL_HOOK__ === "object") {__REACT_DEVTOOLS_GLOBAL_HOOK__.inject = function() {};}</script><script id="__NEXT_DATA__" type="application/json">{"props":{"pageProps":{"isBlacklistedThreadForAdvertising":false,"queryParameters":{"slugs":["money-advice-1127","budgeting-609","551294-you-lovely-savvy-shoppers-688.html"]},"initialReduxState":{"isMobile":{"isMobile":true},"login":{"token":null,"userId":null,"firstName":null,"lastName":null,"username":null,"userSlug":null,"email":null,"role":null},"currentThread":{"currentThread":{"id":"551294","subject":"To all you lovely savvy shoppers","questionCleanedText":"For all of you who hold your breath at the checkout waiting for the final total of your shopping:shock:  We are a family of 7 myself,OH,3DS's 14,12 and 9 and 2 DD's 7 and 3.  We used to spend £680 a month at Asda and unbeleaveably we'd still have days where we'd say we have'nt got anything for dinner:shock: and then full of shame I must admit we'd buy take-aways so that could be another £200 in a month so £880 in 1 month:shock: .  Now I got so fed up with this:yuk: are shopping not only cost us a fortune it took us flamming ages as well:roll:.  So we swapped to having our shopping delivered once a month from Asda and got it down to £440 + still £200 worth of take-aways so £680 a month.  After coming on here and reading about Meal Planning.A light bulb went on(at last) and we started doing them.We now do one once a month,we all choose meals that we want for dinner.If we don't fancy what's down for one day we just swap it to another day. So not set in stone but it does help to know it's there.  We've now got it down to £260 a month:nmsdance:  We now buy our main shop at Lidl Fruit and vegetables on the market Ice Lollies,Ice Cream,and any great offers(we got box's of cup drinks before for 50p) Toilitries and Cleaning things and some medicines from the 99p shop \u0026nbsp; We do all this once a month,fruit and vegetables we fill the fruit basket and then put some veg into the fridge the rest goes into keep fresh bags.  We spend upto £156 a month in Lidl that includes,Tea bags,coffee,squash(8),Fresh Juice Orange and Apple(56),Fizzy Various ones(12),Crisps(48),Tortilla Chips(3),Salsa(2),Cereal(4)Coissants(4) Beans(8),tinned toms(4), garlic Puree(2), Tomatoe Puree(3),Pasta and dry Spaghetti(1 of each),Tuna(4),corned beef(2),Cooking Sauces(4)5kg Basmati Rice(5 months worth),Biscuits(6),Sweets(to many to count),Bread Flour(8),Yeast Satchets(8),Icing Sugar(1),Mayonnaise(2),tom sauce(2) Bbq sauce(2),Gravy Granuleslarge(1),Stuffing Mix(1)Pull-ups(8),Washing Powder(1) £8 for 100 washes so lasts over a month,large hand soap pumps(2),Toothpaste childrens one (1),Adults Toothpaste(1)Large fresh Mackeral(2),Bags of Pollock(2), Bags ofChicken Breasts(2),Mince 2KGS,Pork loin chops 4 in a packet(2),salami(2),Ham(2),Cheese large blocks(10), large Joint of Gammon(1),Frozen Broccoli(3)Frozen Cauliflower(2).  Market 2 large bags of Bannana's (60) 4 large bags of Apples(40 2 large bags of Plums(24) 2 large bags of Grapes(£9 worth for £1.50) 4 large punnets of Strawberries(£10 worth for £2) 4 large Mangos(£4.50 worth for 80p) 2 large bags of Nectarines(20) 2 large bags of Peaches(20) 1 large bag of Mushrooms(40) 2 large bags of Carrotts(60) 3 Romaine Lettuces 2 large Cuecumbers 1 Large Cabbage(3 dinners worth) 1 large White Cabbage(to stirfry with and make coleslaw with) That should be about £28 but they allways knock money off for regular customers so £20  Iceland Icelollies(60+) 4LTR Icecream 12 Frozen DoughNuts 8 Chocolate Eclairs Ice Cream Cones 2 Box's of Chocolate Flakes Ice Cream Sauces(4 every 2 months) sprinkles(6 every 6 months) Quater Pounders Lamb(8),Beef(8) Beef large one(2 dinners) Lamb(1 dinner) Sausages (60) used for dinners,lunch and breakfast. Hotdog Rolls(24) Burger Rolls(48)used for Burger Dinners,Lunches for rolls with fillings,Rolls made into Pizzas. Frozen garden Peas large bag(1) Frozen corn on the cobs 6 in a bag(3) Frozen Peppers(1) Frozen Parsnips(1) Yogurts(24) Chocolate Mousses(8) Large Spray Cream(1) Crumpets(24) Tea cakes(16)( these 3 products we allways manage to get reduced for about 25p a packet) Muffins(24) About £57 a month.  99p Store 1 Shampoo(got a huge salon one)Both of these last 2 months so 392 hair washes 1 Conditioner(got a huge salon one) 9 packs of Toilet Rolls(54 toilet rolls) 2 packs of Kitchen Towels(4 rolls) 2 large bottles of Washing up liquid(99p) 1 large Bleach Deodrants Various ones for 5 of us Black Bags 2(20 in a pack) Soap(4 bars every 2 months) 2 large Bubble baths about £20.  We go to Asda once every 6 months and get 3 Smartprice Polishes 2 large Bathroom/Kitchen Cleaners Packets of Razors 6 Cans Of Shaving Foam 4 Showergels 6 Large bags of S/R Flour and 6 Large Bags of Plain Flour Large Bag of Sultana's and Large Bag of Coconut. 6 Large Bags of Cooking Chocolate Probably about £23 every 6 months(smartprice products except for showergel and cooking chocolate) \u0026nbsp; We get our eggs 30 free range a month from a local shop for £2.40 And a Huge sack of Onions for £1.49 that'll last about 3-4 months.  So that's all our Breakfasts,Lunchs,Dinners,Suppers,Snacks,Treats,Drinks,Toilitries and Cleaning Stuff for all 7 of us a month.  We make our own Nan Breads and Flour Tortillas as they cost pennys to make,our Own Bread and Lots of our Own Puddings and Cakes.  OH has now got a Dab Hand at making his own Fake -Aways.So we rarely have a Take -away now:D  Our Diet was allways good but now it's even better and far more varied. Our shopping now only takes us 2hrs a month to do it used to be 6hrs a month.  And the biggest difference is we used to spend £8,160 a year on our food shop and now it's £3,120 a year on our food shop so we save £5,040 a year now.Thats not including the Take-aways now were not spending £200 a month on them that's probably an extra £2,100 saved a year so all in £7,140 saved a year:shock: .  So if you need some ideas or inspiration for slashing your food shop because you want to or you really need to I hope something here helps. :hug:xx","isClosed":false,"isSponsored":false,"postsCount":14300,"user":{"avatarUrl":"","pseudo":"Anonymous","userSlug":"","registrationDate":"","postedMessagesCount":0,"isOnline":false,"anonymized":true,"signature":"","role":""},"pagePosts":[{"id":"8905360","user":{"avatarUrl":"","pseudo":"Anonymous","userSlug":"","registrationDate":"","postedMessagesCount":0,"isOnline":false,"anonymized":true,"signature":"","role":""},"date":"2013-02-18T10:25","content":"\u003cimg title=\":hiya:\" alt=\":hiya:\" src=\"https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/greet024.gif\" class=\"wysiwyg_smiley\" \u003e Morning ladies. \u003cbr\u003e\u003cbr\u003eMe and Ryan slept downstairs last night as OH has a chest infection and was coughing and spluttering all night. \u003cbr\u003e\u003cbr\u003eAndrea hope you're okay xx\u003cbr\u003e\u003cbr\u003eClaire I found when I cut fizzy drinks out I also stopped with so many headaches. We're in the area that starts the universal credits in April. Apparently it will make families no worse off, it's just compiling payments to monthly. This is what we have been told anyway that we're not going to be worse off. Hopefully anyway! \u003cbr\u003e\u003cbr\u003eCheryl I know how you feel with a bad chest. I only have part function of my right lung after having 8 blood clots on my lung and pneumonia at the same time ever since I have not been right and that was November 2009. Nothing worse than a sore chest. Have you tried sleeping sat upright? I found this helped and for 8 months after getting ill I slept downstairs sat up in a chair. \u003cbr\u003e\u003cbr\u003eNot got much planned today. Need to clean up and tidy round. OH has got football tonight, even though he is ill. They're playing reading so he's looking forward to that.\u003cbr\u003e\u003cbr\u003eDS1's knee is a lot better this morning but looks horrendous when I changed the dressing \u003cimg title=\":yuk:\" alt=\":yuk:\" src=\"https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/yuk.gif\" class=\"wysiwyg_smiley\" \u003e\u003cbr\u003e\u003cbr\u003eWe have lovely blue sky today and it's warmish out but with a fresh feeling to the air. Sold more clothes on ebay last night, don't have to deal with postage as she is collecting them. Gutted they didn't go for more £4.20 for £130 worth of clothes! Some are brand new next too.","quotedPosts":[],"likesCount":0,"editedAt":"","editedBy":"","computedId":"22_609_551294_8905360"},{"id":"8905415","user":{"avatarUrl":"","pseudo":"Anonymous","userSlug":"","registrationDate":"","postedMessagesCount":0,"isOnline":false,"anonymized":true,"signature":"","role":""},"date":"2013-02-18T10:46","content":"Morning \u003cimg title=\":hiya:\" alt=\":hiya:\" src=\"https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/greet024.gif\" class=\"wysiwyg_smiley\" \u003e\u003cbr\u003e\u003cbr\u003ebeen busy again and so far have done all the ironing. Yet more washing in the machine \u003cimg title=\":roll:\" alt=\":roll:\" src=\"https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/icon_rolleyes.gif\" class=\"wysiwyg_smiley\" \u003e Harry and I cycled into town and back about 2 miles each way so I have done my exercise for today!\u003cbr\u003e\u003cbr\u003eJust going to start making my steak and kidney pies in a bit.\u003cbr\u003e\u003cbr\u003eAlso got to help Harry with a history project on the Normans and he has the Horrible Histories collection so should find some interesting facts in those.\u003cbr\u003eWill sort Harry's room this afternoon when Steve is awake as I don't want to disturb him.\u003cbr\u003e\u003cbr\u003eJust having a nice low cal hot chocolate.\u003cbr\u003e\u003cbr\u003eCheryl - hope you are feeling a bit better today?\u003cbr\u003e\u003cbr\u003eAndrea- I am sure you and Frazer can sort it out, he obviously really loves you and Caitlin. \u003cimg title=\":hug:\" alt=\":hug:\" src=\"https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/hugs.gif\" class=\"wysiwyg_smiley\" \u003e Maybe he is a bit stressed at the moment.\u003cbr\u003e\u003cbr\u003eSharon- hope the mole is ok and like the sound of your bargains.\u003cbr\u003e\u003cbr\u003eOff to start helping Harry and start cooking.\u003cbr\u003eHave a good day everyone xx","quotedPosts":[],"likesCount":0,"editedAt":"","editedBy":"","computedId":"22_609_551294_8905415"},{"id":"8905520","user":{"avatarUrl":"https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/user/avatar/default_tns1_avatar.png?v=1613050416","pseudo":"karine j(2)","userSlug":"karine-2","registrationDate":"2008-04-27","postedMessagesCount":347,"isOnline":false,"anonymized":false,"signature":"","role":""},"date":"2013-02-18T11:19","content":"Claire what is this universal credit?\u003cbr\u003e\u003cbr\u003eAndrea I hope Frazer is back to himlself soon XXX\u003cbr\u003e\u003cbr\u003eSamantha, hope your OH feels better really soon.\u003cbr\u003e\u003cbr\u003eDawn, would you have some recipes for healthy fish cakes? \u003cimg title=\";)\" alt=\";)\" src=\"https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/icon_wink.gif\" class=\"wysiwyg_smiley\" \u003e I know you have most recipes.","quotedPosts":[],"likesCount":0,"editedAt":"","editedBy":"","computedId":"22_609_551294_8905520"},{"id":"8905523","user":{"avatarUrl":"","pseudo":"Anonymous","userSlug":"","registrationDate":"","postedMessagesCount":0,"isOnline":false,"anonymized":true,"signature":"","role":""},"date":"2013-02-18T11:19","content":"Hi all\u003cbr\u003e\u003cbr\u003eHugs to those who need them ! \u003cimg title=\":hug:\" alt=\":hug:\" src=\"https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/hugs.gif\" class=\"wysiwyg_smiley\" \u003e\u003cbr\u003e\u003cbr\u003e\u003cbr\u003eJust staying in today, still feel a bit rotten. Have swept the kitchen floor, put some washing away and dusted all of upstairs.\u003cbr\u003e\u003cbr\u003eGot an online sainsburys shop coming after lunch so DS can help me put the shopping away later !\u003cbr\u003e\u003cbr\u003eNot much else to report ! xxxx","quotedPosts":[],"likesCount":0,"editedAt":"","editedBy":"","computedId":"22_609_551294_8905523"},{"id":"8905953","user":{"avatarUrl":"","pseudo":"Anonymous","userSlug":"","registrationDate":"","postedMessagesCount":0,"isOnline":false,"anonymized":true,"signature":"","role":""},"date":"2013-02-18T12:49","content":"Hi everyone.\u003cbr\u003eAndrea, sometimes men need a break, they don't function like us mummies. A few days at his mums might do him good and help recharge his batteries. Xx\u003cbr\u003e\u003cbr\u003eElise, what picture are you making? \u003cbr\u003e\u003cbr\u003eSam I sympathise Hun I only have 40% of my right lung that works. I have no choice than to sleep sitting up when I'm like this. I've had brittle asthma for years. You get used to it but it never seems to get easier lol. \u003cbr\u003e\u003cbr\u003eSharon I hope you don't the hypochondriac look I seem to get lately. Sorry to hear your poorly Hun. Get better soon xxx\u003cbr\u003e\u003cbr\u003eDawn, how are you today lovely? \u003cbr\u003e\u003cbr\u003eClaire not sure if it helps but for meatballs I buy mince add Oxo and onions and make into balls then add either chopped tomatoes or passata a little garlic, tomato purée and Italian herbs. Fry off the meatballs first with extra chopped onion and peppers then add the rest. Bring to the boil then simmer until cooked. I serve either alone or cook pasta and place pasta in a dish then top with the meatballs and sauce and top that with cheese then bake at about 180 until cheese bubbling. \u003cbr\u003e\u003cbr\u003eMichelle, put your feet up after shopping arrives. Xx\u003cbr\u003e\u003cbr\u003eDebbie, what are you cooking? \u003cbr\u003e\u003cbr\u003eKarine, I have black slate in my kitchen and it drives me mad. How's the weather there? \u003cbr\u003e\u003cbr\u003eSusan, what sort of auction was it? I love a good sale.\u003cbr\u003e\u003cbr\u003eJade, can you come and do my garden? It's such a mess. Need to get out there once the warmer weather comes and I'm feeling better. \u003cbr\u003e\u003cbr\u003e\u003cbr\u003eSorry if I've missed anyone. Where is Kayleigh? Not seen her on here for a bit, or angel. Wonder if she's had the baby yet??\u003cbr\u003e\u003cbr\u003eI'm off to make some lunch for ds then try to rest a bit. Steroids are really helping today but I still get short of breath with any exertion.","quotedPosts":[],"likesCount":0,"editedAt":"","editedBy":"","computedId":"22_609_551294_8905953"},{"id":"8905984","user":{"avatarUrl":"https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/user/avatar/default_tns1_avatar.png?v=1613050416","pseudo":"Jennie W(29)","userSlug":"jennie-w-29","registrationDate":"2008-01-27","postedMessagesCount":0,"isOnline":false,"anonymized":false,"signature":"DS- 07/06/05\n\nDD-18/02/09","role":""},"date":"2013-02-18T12:54","content":"Afternoon:hiya:\u003cbr\u003e\u003cbr\u003eWell it's Phoebe's 4th bday! We bought her a Barbie Head with hair accessories \u0026amp; a Hello Kitty Drawing Case (so she doesn't pinch Lewis';))Got her a floor jigsaw from Lewis, SS got her a load of colouring things that she will enjoy doing then my dad got her a Little Pony. \u0026nbsp;I put a status on FB \u0026amp; everyone's commented! We are having a little tea party for her so I have spent all morning making chilli, making tuna \u0026amp; egg mayo (will make the sandwiches in an hour or so!), cleaning up from the mess I made:lol: then made some choc rice crispies buns (\u0026amp; cleaned up from that mess:roll:) Just need to clean up from dinner. Also done a towel wash \u0026amp; will hang that out after this then need to clean the bathroom while I'm up there! \u003cbr\u003e\u003cbr\u003eAndrea-Sorry to hear about you \u0026amp; Frazer. Your'e more than welcome to PM me xxx\u003cbr\u003e\u003cbr\u003eYesterday spent £77 in Morrisons on bits for the party, still need to get some meat but will get sent Stu up for some on Wednesday! He's out \u0026amp; about so hoping he'll be home soon! Kids are playing on the Wii while I'm trying to tidy the house:roll: Have cleaned the porch so will spray it with air freshener before everyone turns up!\u003cbr\u003e\u003cbr\u003eRight better get on with tidying the bathroom x","quotedPosts":[],"likesCount":0,"editedAt":"","editedBy":"","computedId":"22_609_551294_8905984"},{"id":"8906012","user":{"avatarUrl":"","pseudo":"Anonymous","userSlug":"","registrationDate":"","postedMessagesCount":0,"isOnline":false,"anonymized":true,"signature":"","role":""},"date":"2013-02-18T13:00","content":"cheryl.... the auction is an outdoor one, just a load of crap most weeks lol, but i did get OH an old 2gallon petrol can one week for £7.... as he collects them got way too many now tho that we need to sell them as we want to move to a bungalow as OH has a back condition thats getting worse....","quotedPosts":[],"likesCount":0,"editedAt":"","editedBy":"","computedId":"22_609_551294_8906012"},{"id":"8906243","user":{"avatarUrl":"","pseudo":"Anonymous","userSlug":"","registrationDate":"","postedMessagesCount":0,"isOnline":false,"anonymized":true,"signature":"","role":""},"date":"2013-02-18T13:46","content":"\u003cbr\u003e\u003cbr\u003e\u003cbr\u003eUp until this year the war and peace show has been hosted not far from me. They sell hundreds of these there! Your oh would be in his element lol.\u003cbr\u003e\u003cbr\u003eJennie, happy birthday to phoebe. I hope you all have a lovely day. X","quotedPosts":[{"quotedPostAuthor":{"avatarUrl":"","pseudo":"Anonymous","userSlug":"","registrationDate":"","postedMessagesCount":0,"isOnline":false,"anonymized":true,"signature":"","role":""},"quotedPostContent":"cheryl.... the auction is an outdoor one, just a load of crap most weeks lol, but i did get OH an old 2gallon petrol can one week for £7.... as he collects them got way too many now tho that we need to sell them as we want to move to a bungalow as OH has a back condition thats getting worse...."}],"likesCount":0,"editedAt":"","editedBy":"","computedId":"22_609_551294_8906243"},{"id":"8906866","user":{"avatarUrl":"","pseudo":"Anonymous","userSlug":"","registrationDate":"","postedMessagesCount":0,"isOnline":false,"anonymized":true,"signature":"","role":""},"date":"2013-02-18T16:06","content":"Jennie - Happy birthday to Phoebe! :partytime::birthday: Hope you have a lovely tea party x\u003cbr\u003e\u003cbr\u003eCheryl - glad that you are feeling a bit better today. Making HM Steak and kidney pies. Meat etc in slow cooker and I am going to make the pies with pastry topping in a minute and serve with mashed potatoes and veg. Boys having something different as they don't like it.\u003cbr\u003e\u003cbr\u003eHad a bit of a rest this afternoon. Like being at home \u003cimg title=\":D\" alt=\":D\" src=\"https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/icon_biggrin.gif\" class=\"wysiwyg_smiley\" \u003e","quotedPosts":[],"likesCount":0,"editedAt":"","editedBy":"","computedId":"22_609_551294_8906866"},{"id":"8907311","user":{"avatarUrl":"","pseudo":"Anonymous","userSlug":"","registrationDate":"","postedMessagesCount":0,"isOnline":false,"anonymized":true,"signature":"","role":""},"date":"2013-02-18T18:02","content":"Cheryl your meatball recipe is great, and putting it all in one pot on the oven sounds tasty too!\u003cbr\u003e\u003cbr\u003eKarine, universal credit is the new benefit system. It means all benefits will be under one name. EG if you get housing benefit and jsa, it will all come in one payment and housing ben wont be paid by the local council it will all be paidf by government I think. \u003cbr\u003e\u003cbr\u003eSamantha I didnt realised it compiled all payments to monthly! We only get tax credits but recenbtly got them changed to weekly which has been much better, been able to keep on top of our food shopping and means if dh gets called to work (he is a supply lecturer) we wouldnt have to worry about having petrol money. It will be a real struggle to get food shopping if we go back to onthly \u003cimg title=\":(\" alt=\":(\" src=\"https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/icon_sad.gif\" class=\"wysiwyg_smiley\" \u003e\u003cbr\u003e\u003cbr\u003e\u003cbr\u003eJennie - happy birthday Pheobe!\u003cbr\u003e\u003cbr\u003eBIL gone to pub with a friend, just me and the kids. theyre having dinner, mines is cooking then going for a shower and put my jammies on. Work was manic today, i dont know what everyone else was doing but I saw 12 patients and done 10 sales. Felt like I never stopped all day!","quotedPosts":[],"likesCount":0,"editedAt":"","editedBy":"","computedId":"22_609_551294_8907311"},{"id":"8908156","user":{"avatarUrl":"","pseudo":"Anonymous","userSlug":"","registrationDate":"","postedMessagesCount":0,"isOnline":false,"anonymized":true,"signature":"","role":""},"date":"2013-02-18T20:52","content":"\u003cbr\u003e\u003cbr\u003e\u003cbr\u003ewhere abouts is that?.. god he would be like a kid in a sweet shop lol","quotedPosts":[{"quotedPostAuthor":{"avatarUrl":"","pseudo":"Anonymous","userSlug":"","registrationDate":"","postedMessagesCount":0,"isOnline":false,"anonymized":true,"signature":"","role":""},"quotedPostContent":"\u003cbr /\u003e\u003cbr</script></body></html>
"""

In [26]:
soup=BeautifulSoup(html, "html5lib")

In [89]:
t = """{"test":"\":yuck:\""}"""
res = re.sub(pattern,r'\\"\1\\"',t)
json.loads(t)

JSONDecodeError: Expecting ',' delimiter: line 1 column 11 (char 10)

In [102]:
import json
imgs = soup.find_all("img", {"class":"wysiwyg_smiley"})
for img in imgs:
    if img.has_attr("title"):
        replacement = " " + img.get("title") + " "
    else:
        replacement = " ::emoji:: "
    img.replace_with(replacement)
script=soup.find("script", {"id":"__NEXT_DATA__"})
# script=script.get_text()
print(script)

<script id="__NEXT_DATA__" type="application/json">{"props":{"pageProps":{"isBlacklistedThreadForAdvertising":false,"queryParameters":{"slugs":["money-advice-1127","budgeting-609","551294-you-lovely-savvy-shoppers-688.html"]},"initialReduxState":{"isMobile":{"isMobile":true},"login":{"token":null,"userId":null,"firstName":null,"lastName":null,"username":null,"userSlug":null,"email":null,"role":null},"currentThread":{"currentThread":{"id":"551294","subject":"To all you lovely savvy shoppers","questionCleanedText":"For all of you who hold your breath at the checkout waiting for the final total of your shopping:shock:  We are a family of 7 myself,OH,3DS's 14,12 and 9 and 2 DD's 7 and 3.  We used to spend £680 a month at Asda and unbeleaveably we'd still have days where we'd say we have'nt got anything for dinner:shock: and then full of shame I must admit we'd buy take-aways so that could be another £200 in a month so £880 in 1 month:shock: .  Now I got so fed up with this:yuk: are shoppin

In [98]:
pattern=re.compile(r'\":([a-z]*):\"', re.IGNORECASE)
fixed = re.sub(pattern,r'\\\"~\1~\\\"',script)
print(fixed)

{"props":{"pageProps":{"isBlacklistedThreadForAdvertising":false,"queryParameters":{"slugs":["money-advice-1127","budgeting-609","551294-you-lovely-savvy-shoppers-688.html"]},"initialReduxState":{"isMobile":{"isMobile":true},"login":{"token":null,"userId":null,"firstName":null,"lastName":null,"username":null,"userSlug":null,"email":null,"role":null},"currentThread":{"currentThread":{"id":"551294","subject":"To all you lovely savvy shoppers","questionCleanedText":"For all of you who hold your breath at the checkout waiting for the final total of your shopping:shock:  We are a family of 7 myself,OH,3DS's 14,12 and 9 and 2 DD's 7 and 3.  We used to spend £680 a month at Asda and unbeleaveably we'd still have days where we'd say we have'nt got anything for dinner:shock: and then full of shame I must admit we'd buy take-aways so that could be another £200 in a month so £880 in 1 month:shock: .  Now I got so fed up with this:yuk: are shopping not only cost us a fortune it took us flamming ag

In [99]:
fixed=json.loads(fixed)

JSONDecodeError: Expecting ',' delimiter: line 1 column 6412 (char 6411)

In [42]:
text = '''{"content":"<img title=":hiya:" alt=":hiya:" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/greet024.gif" class="wysiwyg_smiley" > Morning ladies. <br><br>Me and Ryan slept downstairs last night as OH has a chest infection and was coughing and spluttering all night. <br><br>Andrea hope you're okay xx<br><br>Claire I found when I cut fizzy drinks out I also stopped with so many headaches. We're in the area that starts the universal credits in April. Apparently it will make families no worse off, it's just compiling payments to monthly. This is what we have been told anyway that we're not going to be worse off. Hopefully anyway! <br><br>Cheryl I know how you feel with a bad chest. I only have part function of my right lung after having 8 blood clots on my lung and pneumonia at the same time ever since I have not been right and that was November 2009. Nothing worse than a sore chest. Have you tried sleeping sat upright? I found this helped and for 8 months after getting ill I slept downstairs sat up in a chair. <br><br>Not got much planned today. Need to clean up and tidy round. OH has got football tonight, even though he is ill. They're playing reading so he's looking forward to that.<br><br>DS1's knee is a lot better this morning but looks horrendous when I changed the dressing <img title=":yuk:" alt=":yuk:" src="https://www.netmums.com/coffeehouse/profile/static/generated/merged/flat/design/smilies/yuk.gif" class="wysiwyg_smiley" ><br><br>We have lovely blue sky today and it's warmish out but with a fresh feeling to the air. Sold more clothes on ebay last night, don't have to deal with postage as she is collecting them. Gutted they didn't go for more £4.20 for £130 worth of clothes! Some are brand new next too."}'''

In [43]:
json.loads(text)

JSONDecodeError: Expecting ',' delimiter: line 1 column 25 (char 24)